In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from scipy import signal
import sys
import math
import pickle
from tabulate import tabulate

In [ ]:
#aktuell beste:
Parameter = {"xtick.labelsize": 22, "ytick.labelsize": 22, "legend.fontsize": 27, "axes.titlesize": 32, "axes.labelsize": 32,
            "lines.linewidth": 2, "scatter.marker": "o", "figure.figsize": [12, 9]}

plt.rcParams.update(Parameter)

#################  Mean Trajectory Analysis ########################

In [ ]:
def rescaleCoordinates1to5(condMean):
    condMean[:,0] = condMean[:,0]-condMean[0,0]
    condMean[:,1] = condMean[:,1]-condMean[0,1]
    condMean[:,2] = condMean[:,2]-condMean[0,2]

    return condMean


In [ ]:
def rescaleCoordinates6to10(condMean):
    condMean[:,0] = (condMean[:,0]-condMean[0,0])*-1
    condMean[:,1] = condMean[:,1]-condMean[0,1]
    condMean[:,2] = (condMean[:,2]-condMean[0,2])*-1

    return condMean

In [ ]:
def groupMean(condMeanP1, condMeanP2, condMeanP3, condMeanP4, condMeanP5, condMeanP6, condMeanP7, condMeanP8, condMeanP9, condMeanP10):
    xMeans = np.column_stack((condMeanP1[:,0], condMeanP2[:,0], condMeanP3[:,0], condMeanP4[:,0], condMeanP5[:,0], condMeanP6[:,0], condMeanP7[:,0], condMeanP8[:,0], condMeanP9[:,0], condMeanP10[:,0]))
    zMeans = np.column_stack((condMeanP1[:,1], condMeanP2[:,1], condMeanP3[:,1], condMeanP4[:,1], condMeanP5[:,1], condMeanP6[:,1], condMeanP7[:,1], condMeanP8[:,1], condMeanP9[:,1], condMeanP10[:,1]))
    yMeans = np.column_stack((condMeanP1[:,2], condMeanP2[:,2], condMeanP3[:,2], condMeanP4[:,2], condMeanP5[:,2], condMeanP6[:,2], condMeanP7[:,2], condMeanP8[:,2], condMeanP9[:,2], condMeanP10[:,2]))

    xGroupMean = []
    for i in range(len(xMeans)):
        xGroupMean.append(np.mean(xMeans[i,:]))

    zGroupMean = []
    for i in range(len(zMeans)):
        zGroupMean.append(np.mean(zMeans[i,:]))

    yGroupMean = []
    for i in range(len(yMeans)):
        yGroupMean.append(np.mean(yMeans[i,:]))

    GroupMean = np.array((xGroupMean,yGroupMean,zGroupMean)).T

    return GroupMean

In [ ]:
def groupVariance(condVarianceP1, condVarianceP2, condVarianceP3, condVarianceP4, condVarianceP5, condVarianceP6, condVarianceP7, condVarianceP8, condVarianceP9, condVarianceP10):
    xVariances = np.column_stack((condVarianceP1[:,0], condVarianceP2[:,0], condVarianceP3[:,0], condVarianceP4[:,0], condVarianceP5[:,0], condVarianceP6[:,0], condVarianceP7[:,0], condVarianceP8[:,0], condVarianceP9[:,0], condVarianceP10[:,0]))
    zVariances = np.column_stack((condVarianceP1[:,1], condVarianceP2[:,1], condVarianceP3[:,1], condVarianceP4[:,1], condVarianceP5[:,1], condVarianceP6[:,1], condVarianceP7[:,1], condVarianceP8[:,1], condVarianceP9[:,1], condVarianceP10[:,1]))
    yVariances = np.column_stack((condVarianceP1[:,2], condVarianceP2[:,2], condVarianceP3[:,2], condVarianceP4[:,2], condVarianceP5[:,2], condVarianceP6[:,2], condVarianceP7[:,2], condVarianceP8[:,2], condVarianceP9[:,2], condVarianceP10[:,2]))

    xGroupVariance = []
    for i in range(len(xVariances)):
        xGroupVariance.append(np.mean(xVariances[i,:]))

    zGroupVariance = []
    for i in range(len(zVariances)):
        zGroupVariance.append(np.mean(zVariances[i,:]))

    yGroupVariance = []
    for i in range(len(yVariances)):
        yGroupVariance.append(np.mean(yVariances[i,:]))

    GroupVariance = np.array((xGroupVariance,yGroupVariance,zGroupVariance)).T

    return GroupVariance
    

In [ ]:
def GroupMeanPertOnset(pertOnsetsP1, pertOnsetsP2, pertOnsetsP3, pertOnsetsP4, pertOnsetsP5, pertOnsetsP6, pertOnsetsP7, pertOnsetsP8, pertOnsetsP9):
    meanPertOnsets = np.column_stack((pertOnsetsP1[:,1], pertOnsetsP2[:,1], pertOnsetsP3[:,1], pertOnsetsP4[:,1], pertOnsetsP5[:,1], pertOnsetsP6[:,1], pertOnsetsP7[:,1], pertOnsetsP8[:,1], pertOnsetsP9[:,1]))

    GroupMeanPertOnsets = []
    for i in range(len(meanPertOnsets)):
        GroupMeanPertOnsets.append(np.mean(meanPertOnsets[i,:]))

    return GroupMeanPertOnsets
    

In [ ]:
def getTimeParams():
    dts = []
    PertSteps = []
    for i in range(len(GroupMeanPertOnsets)):
        dts.append(GroupMeanMoveDurations[i+1]/50)
        PertSteps.append(GroupMeanPertOnsets[i]/dts[i])

    print(dts)
    print(PertSteps)



In [ ]:
def GroupMeanMoveDuration(MoveDurationsP1, MoveDurationsP2, MoveDurationsP3, MoveDurationsP4, MoveDurationsP5, MoveDurationsP6, MoveDurationsP7, MoveDurationsP8, MoveDurationsP9, MoveDurationsP10):
    meanMoveDurations = np.column_stack((MoveDurationsP1[:,1], MoveDurationsP2[:,1], MoveDurationsP3[:,1], MoveDurationsP4[:,1], MoveDurationsP5[:,1], MoveDurationsP6[:,1], MoveDurationsP7[:,1], MoveDurationsP8[:,1], MoveDurationsP9[:,1], MoveDurationsP10[:,1]))

    GroupMeanMoveDurations = []
    for i in range(len(meanMoveDurations)):
        GroupMeanMoveDurations.append(np.mean(meanMoveDurations[i,:]))

    return GroupMeanMoveDurations
    

In [ ]:
def plotGroupMean(T, condGroupMean, condGroupVariance, condName):

    t = np.linspace(0,T)
    filteredVelocity = signal.savgol_filter(condGroupMean, 7, 1, 0, axis=0)
    errorX = condGroupVariance[:,0]
    errorY = condGroupVariance[:,1]
    errorZ = condGroupVariance[:,2]
    fig = plt.figure()
    #fig.text(0.5, 0.03, 'mean', ha='center', fontsize=Parameter['axes.labelsize'])
    #fig.text(0.03, 0.5, 'hand movement (m)', va='center', rotation='vertical', fontsize=Parameter['axes.labelsize'])
    #plt.title('Mean Velocity')


    plt.subplot(3, 1, 1)
    plt.plot(t,filteredVelocity[:,0],linewidth=2,label="mean velocity", color='black')
    plt.fill_between(t,filteredVelocity[:,0] - errorX, filteredVelocity[:,0]+errorX,linewidth=2.0, color='lightgrey', label='standard deviation')
    #plt.plot(t,filteredVelocity[:,0] + condGroupVariance[:,0],linewidth=2.0,linestyle="--", color='grey')
    #plt.xlabel('time')
    plt.ylabel('forw. vel. (m)')
    plt.xticks([])

    plt.subplot(3, 1, 2)
    plt.plot(t,filteredVelocity[:,1],linewidth=2, color='black')
    plt.fill_between(t,filteredVelocity[:,1] - errorY, filteredVelocity[:,1]+errorY,linewidth=2.0, color='lightgrey')
    #plt.xlabel('time')
    plt.ylabel('lat. vel.')
    plt.xticks([])
    #plt.axvline(x = groupMeanPertOnset, linestyle = '--', color = 'grey')

    plt.subplot(3, 1, 3)
    plt.plot(t,filteredVelocity[:,2],linewidth=2, color='black')
    plt.fill_between(t,filteredVelocity[:,2] - errorZ, filteredVelocity[:,2]+errorZ,linewidth=2.0, color='lightgrey')
    plt.xlabel('path length (\%)')
    plt.ylabel('vert. vel.')

    fig.legend(loc='lower right',bbox_to_anchor=(0.5, 0.9))

    plt.savefig('.../' + condName + ".pdf")



In [ ]:
def groupMean(condMeanP1, condMeanP2, condMeanP3, condMeanP4, condMeanP5, condMeanP6, condMeanP7, condMeanP8, condMeanP9, condMeanP10):
    xMeans = np.column_stack((condMeanP1[:,0], condMeanP2[:,0], condMeanP3[:,0], condMeanP4[:,0], condMeanP5[:,0], condMeanP6[:,0], condMeanP7[:,0], condMeanP8[:,0], condMeanP9[:,0], condMeanP10[:,0]))
    zMeans = np.column_stack((condMeanP1[:,1], condMeanP2[:,1], condMeanP3[:,1], condMeanP4[:,1], condMeanP5[:,1], condMeanP6[:,1], condMeanP7[:,1], condMeanP8[:,1], condMeanP9[:,1], condMeanP10[:,1]))
    yMeans = np.column_stack((condMeanP1[:,2], condMeanP2[:,2], condMeanP3[:,2], condMeanP4[:,2], condMeanP5[:,2], condMeanP6[:,2], condMeanP7[:,2], condMeanP8[:,2], condMeanP9[:,2], condMeanP10[:,2]))

    xGroupMean = []
    for i in range(len(xMeans)):
        xGroupMean.append(np.mean(xMeans[i,:]))

    zGroupMean = []
    for i in range(len(zMeans)):
        zGroupMean.append(np.mean(zMeans[i,:]))

    yGroupMean = []
    for i in range(len(yMeans)):
        yGroupMean.append(np.mean(yMeans[i,:]))

    GroupMean = np.array((xGroupMean,yGroupMean,zGroupMean)).T

    return GroupMean

In [ ]:
def TotalMean(cond1Mean, cond2Mean, cond3Mean, cond4Mean, cond5Mean, cond6Mean, cond7Mean, cond8Mean, cond9Mean, cond10Mean, cond11Mean, cond12Mean, cond13Mean):
    xMeans = np.column_stack((cond1Mean[:,0], cond2Mean[:,0], cond3Mean[:,0], cond4Mean[:,0], cond5Mean[:,0], cond6Mean[:,0], cond7Mean[:,0], cond8Mean[:,0], cond9Mean[:,0], cond10Mean[:,0], cond11Mean[:,0 ], cond12Mean[:,0 ], cond13Mean[:, 0]))
    zMeans = np.column_stack((cond1Mean[:,1], cond2Mean[:,1], cond3Mean[:,1], cond4Mean[:,1], cond5Mean[:,1], cond6Mean[:,1], cond7Mean[:,1], cond8Mean[:,1], cond9Mean[:,1], cond10Mean[:,1], cond11Mean[:,1 ], cond12Mean[:,1 ], cond13Mean[:, 1]))
    yMeans = np.column_stack((cond1Mean[:,2], cond2Mean[:,2], cond3Mean[:,2], cond4Mean[:,2], cond5Mean[:,2], cond6Mean[:,2], cond7Mean[:,2], cond8Mean[:,2], cond9Mean[:,2], cond10Mean[:,2], cond11Mean[:,2 ], cond12Mean[:,2 ], cond13Mean[:, 2]))

    xGroupMean = []
    for i in range(len(xMeans)):
        xGroupMean.append(np.mean(xMeans[i,:]))

    zGroupMean = []
    for i in range(len(zMeans)):
        zGroupMean.append(np.mean(zMeans[i,:]))

    yGroupMean = []
    for i in range(len(yMeans)):
        yGroupMean.append(np.mean(yMeans[i,:]))

    GroupMean = np.array((xGroupMean,yGroupMean,zGroupMean)).T

    return GroupMean

In [ ]:
def plotGroupMeanALL(T, cond2GroupMean, cond3GroupMean, cond4GroupMean, cond5GroupMean, cond6GroupMean, cond7GroupMean, cond2GroupVariance, cond3GroupVariance, cond4GroupVariance, cond5GroupVariance, cond6GroupVariance, cond7GroupVariance, condName):
    fig = plt.figure(0)
    plt.title('Mean Trajectory')
   
    fig.text(0.5, 0.03, 'path length (\%)', ha='center', fontsize=Parameter['axes.labelsize'])
    fig.text(0.03, 0.5, 'hand velocity (m/s)', va='center', rotation='vertical', fontsize=Parameter['axes.labelsize'])
    
    t = np.linspace(0,T)
    filteredTrajectory2 = signal.savgol_filter(cond2GroupMean, 7, 1, 0, axis=0)
    filteredTrajectory3 = signal.savgol_filter(cond3GroupMean, 7, 1, 0, axis=0)
    filteredTrajectory4 = signal.savgol_filter(cond4GroupMean, 7, 1, 0, axis=0)
    filteredTrajectory5 = signal.savgol_filter(cond5GroupMean, 7, 1, 0, axis=0)
    filteredTrajectory6 = signal.savgol_filter(cond6GroupMean, 7, 1, 0, axis=0)
    filteredTrajectory7 = signal.savgol_filter(cond7GroupMean, 7, 1, 0, axis=0)
    errorX2 = cond2GroupVariance[:,0]
    errorY2 = cond2GroupVariance[:,1]
    errorZ2 = cond2GroupVariance[:,2]
    errorX3 = cond3GroupVariance[:,0]
    errorY3 = cond3GroupVariance[:,1]
    errorZ3 = cond3GroupVariance[:,2]
    errorX4 = cond4GroupVariance[:,0]
    errorY4 = cond4GroupVariance[:,1]
    errorZ4 = cond4GroupVariance[:,2]
    errorX5 = cond5GroupVariance[:,0]
    errorY5 = cond5GroupVariance[:,1]
    errorZ5 = cond5GroupVariance[:,2]
    errorX6 = cond6GroupVariance[:,0]
    errorY6 = cond6GroupVariance[:,1]
    errorZ6 = cond6GroupVariance[:,2]
    errorX7 = cond7GroupVariance[:,0]
    errorY7 = cond7GroupVariance[:,1]
    errorZ7 = cond7GroupVariance[:,2]
    
    ax1 = plt.subplot2grid((6,3),(0,0))
    ax1.plot(t,filteredTrajectory2[:,0],linewidth=2, color='black', label='mean velocity (m/s)')
    plt.fill_between(t,filteredTrajectory2[:,0] - errorX2, filteredTrajectory2[:,0]+errorX2,linewidth=2.0, color='lightgrey', label='standard deviation (m/s)')
    plt.xticks([])
    

    ax2 = plt.subplot2grid((6,3),(0,1))
    ax2.plot(t,filteredTrajectory2[:,1],linewidth=2, color='black')
    plt.fill_between(t,filteredTrajectory2[:,1] - errorY2, filteredTrajectory2[:,1]+errorY2,linewidth=2.0, color='lightgrey')
    plt.xticks([])
    
    ax3 = plt.subplot2grid((6,3),(0,2))
    ax3.plot(t,filteredTrajectory2[:,2],linewidth=2, color='black')
    plt.fill_between(t,filteredTrajectory2[:,2] - errorZ2, filteredTrajectory2[:,2]+errorZ2,linewidth=2.0, color='lightgrey')

    ax4 = plt.subplot2grid((6,3),(1,0), sharex = ax1)
    ax4.plot(t,filteredTrajectory3[:,0],linewidth=2, color='black')
    plt.fill_between(t,filteredTrajectory3[:,0] - errorX3, filteredTrajectory3[:,0]+errorX3,linewidth=2.0, color='lightgrey')

    ax5 = plt.subplot2grid((6,3),(1,1))
    ax5.plot(t,filteredTrajectory3[:,1],linewidth=2, color='black')
    plt.fill_between(t,filteredTrajectory3[:,1] - errorY3, filteredTrajectory3[:,1]+errorY3,linewidth=2.0, color='lightgrey')
    plt.xticks([])

    ax6 = plt.subplot2grid((6,3),(1,2))
    ax6.plot(t,filteredTrajectory3[:,2],linewidth=2, color='black')
    plt.fill_between(t,filteredTrajectory3[:,2] - errorZ3, filteredTrajectory3[:,2]+errorZ3,linewidth=2.0, color='lightgrey')
    
    ax7 = plt.subplot2grid((6,3),(2,0), sharex = ax1)
    ax7.plot(t,filteredTrajectory4[:,0],linewidth=2, color='black')
    plt.fill_between(t,filteredTrajectory4[:,0] - errorX4, filteredTrajectory4[:,0]+errorX4,linewidth=2.0, color='lightgrey')
    plt.xticks([])
    
    ax8 = plt.subplot2grid((6,3),(2,1))
    ax8.plot(t,filteredTrajectory4[:,1],linewidth=2, color='black')
    plt.fill_between(t,filteredTrajectory4[:,1] - errorY4, filteredTrajectory4[:,1]+errorY4,linewidth=2.0, color='lightgrey')
    plt.xticks([])

    ax9 = plt.subplot2grid((6,3),(2,2))
    ax9.plot(t,filteredTrajectory4[:,2],linewidth=2, color='black')
    plt.fill_between(t,filteredTrajectory4[:,2] - errorZ4, filteredTrajectory4[:,2]+errorZ4,linewidth=2.0, color='lightgrey')

    ax10 = plt.subplot2grid((6,3),(3,0), sharex = ax1)
    ax10.plot(t,filteredTrajectory5[:,0],linewidth=2, color='black')
    plt.fill_between(t,filteredTrajectory5[:,0] - errorX5, filteredTrajectory5[:,0]+errorX5,linewidth=2.0, color='lightgrey')
    plt.xticks([])
    

    ax11 = plt.subplot2grid((6,3),(3,1))
    ax11.plot(t,filteredTrajectory5[:,1],linewidth=2, color='black')
    plt.fill_between(t,filteredTrajectory5[:,1] - errorY5, filteredTrajectory5[:,1]+errorY5,linewidth=2.0, color='lightgrey')
    plt.xticks([])

    ax12 = plt.subplot2grid((6,3),(3,2))
    ax12.plot(t,filteredTrajectory5[:,2],linewidth=2, color='black')
    plt.fill_between(t,filteredTrajectory5[:,2] - errorZ5, filteredTrajectory5[:,2]+errorZ5,linewidth=2.0, color='lightgrey')

    ax13 = plt.subplot2grid((6,3),(4,0), sharex = ax1)
    ax13.plot(t,filteredTrajectory6[:,0],linewidth=2, color='black')
    plt.fill_between(t,filteredTrajectory6[:,0] - errorX6, filteredTrajectory6[:,0]+errorX6,linewidth=2.0, color='lightgrey')
    plt.xticks([])
    

    ax14 = plt.subplot2grid((6,3),(4,1))
    ax14.plot(t,filteredTrajectory6[:,1],linewidth=2, color='black')
    plt.fill_between(t,filteredTrajectory6[:,1] - errorY6, filteredTrajectory6[:,1]+errorY6,linewidth=2.0, color='lightgrey')
    plt.xticks([])

    ax15 = plt.subplot2grid((6,3),(4,2))
    ax15.plot(t,filteredTrajectory6[:,2],linewidth=2, color='black')
    plt.fill_between(t,filteredTrajectory6[:,2] - errorZ6, filteredTrajectory6[:,2]+errorZ6,linewidth=2.0, color='lightgrey')

    ax16 = plt.subplot2grid((6,3),(5,0), sharex = ax1)
    ax16.plot(t,filteredTrajectory7[:,0],linewidth=2, color='black')
    plt.fill_between(t,filteredTrajectory7[:,0] - errorX7, filteredTrajectory7[:,0]+errorX7,linewidth=2.0, color='lightgrey')
    plt.xticks([])
    

    ax17 = plt.subplot2grid((6,3),(5,1))
    ax17.plot(t,filteredTrajectory7[:,1],linewidth=2, color='black')
    plt.fill_between(t,filteredTrajectory7[:,1] - errorY7, filteredTrajectory7[:,1]+errorY7,linewidth=2.0, color='lightgrey')
    plt.xticks([])

    ax18 = plt.subplot2grid((6,3),(5,2))
    ax18.plot(t,filteredTrajectory7[:,2],linewidth=2, color='black')
    plt.fill_between(t,filteredTrajectory7[:,2] - errorZ7, filteredTrajectory7[:,2]+errorZ7,linewidth=2.0, color='lightgrey')

    fig.legend(loc='upper left',bbox_to_anchor=(0.08, 1.05))
    plt.savefig('.../' + condName + ".pdf")



In [ ]:
def plotGroupMeanCond1(T, condGroupMean, condGroupVariance, condName):

    t = np.linspace(0,T)
    filteredTrajectory = signal.savgol_filter(condGroupMean, 7, 1, 0, axis=0)
    errorX = condGroupVariance[:,0]
    errorY = condGroupVariance[:,1]
    errorZ = condGroupVariance[:,2]
    plt.figure()
    plt.title('Mean Trajectory')


    plt.subplot(3, 1, 1)
    plt.plot(t,filteredTrajectory[:,0],linewidth=2,label="Mean Trajectory", color='black')
    plt.fill_between(t,filteredTrajectory[:,0] - errorX, filteredTrajectory[:,0]+errorX,linewidth=2.0, color='lightgrey')
    plt.ylabel('forw. pos. (m)')
    plt.xticks([])
    plt.legend()

    plt.subplot(3, 1, 2)
    plt.plot(t,filteredTrajectory[:,1],linewidth=2,label="Mean Trajectory", color='black')
    plt.fill_between(t,filteredTrajectory[:,1] - errorY, filteredTrajectory[:,1]+errorY,linewidth=2.0, color='lightgrey')
    plt.ylabel('lat. pos.')
    plt.xticks([])

    plt.subplot(3, 1, 3)
    plt.plot(t,filteredTrajectory[:,2],linewidth=2,label="Mean Trajectory", color='black')
    plt.fill_between(t,filteredTrajectory[:,2] - errorZ, filteredTrajectory[:,2]+errorZ,linewidth=2.0, color='lightgrey', label='standard deviation')
    plt.xlabel('path length (\%)')
    plt.ylabel('vert. pos.')

    plt.savefig('.../' + condName + ".pdf")



In [ ]:
def plotGroupMean(T, condGroupMean, condGroupVariance, condName):

    t = np.linspace(0,T)
    filteredTrajectory = signal.savgol_filter(condGroupMean, 7, 1, 0, axis=0)
    errorX = condGroupVariance[:,0]
    errorY = condGroupVariance[:,1]
    errorZ = condGroupVariance[:,2]
    plt.figure()
    plt.title('Mean Trajectory')


    plt.subplot(3, 1, 1)
    plt.plot(t,filteredTrajectory[:,0],linewidth=2,label="Mean Trajectory", color='black')
    plt.fill_between(t,filteredTrajectory[:,0] - errorX, filteredTrajectory[:,0]+errorX,linewidth=2.0, color='lightgrey')
    plt.ylabel('forw. pos. (m)')
    plt.xticks([])
    plt.legend()

    plt.subplot(3, 1, 2)
    plt.plot(t,filteredTrajectory[:,1],linewidth=2,label="Mean Trajectory", color='black')
    plt.fill_between(t,filteredTrajectory[:,1] - errorY, filteredTrajectory[:,1]+errorY,linewidth=2.0, color='lightgrey')
    plt.ylabel('lat. pos.')
    plt.xticks([])

    plt.subplot(3, 1, 3)
    plt.plot(t,filteredTrajectory[:,2],linewidth=2,label="Mean Trajectory", color='black')
    plt.fill_between(t,filteredTrajectory[:,2] - errorZ, filteredTrajectory[:,2]+errorZ,linewidth=2.0, color='lightgrey', label='standard deviation')
    plt.xlabel('path length (\%)')
    plt.ylabel('vert. pos.')

    #plt.savefig('.../' + condName + ".pdf")



In [ ]:
def finiteDifferenceHigherOrder(xVel, yVel, zVel):
    
    """ 
        Caclulate xDerivative by 2nd/4th order finite difference method.
        http://www.math.ubc.ca/~jfeng/CHBE553/Example7/Formulae.pdf 
    """

    step = 0.02
    xDerivative = np.empty_like(xVel) 

    xDerivative[:2] = ((-xVel[2:4] + 4*xVel[1:3] - 3*xVel[:2]) / (2*step)) # forward
    xDerivative[-2:] = ((3*xVel[-2:] - 4*xVel[-3:-1] + xVel[-4:-2]) / (2*step)) # backward
    xDerivative[2:-2] = ((-xVel[4:] + 8*(xVel[3:-1] - xVel[1:-3]) + xVel[:-4]) /(12*step)) # central

    yDerivative = np.empty_like(yVel) 

    yDerivative[:2] = ((-yVel[2:4] + 4*yVel[1:3] - 3*yVel[:2]) / (2*step))
    yDerivative[-2:] = ((3*yVel[-2:] - 4*yVel[-3:-1] + yVel[-4:-2]) / (2*step)) 
    yDerivative[2:-2] = ((-yVel[4:] + 8*(yVel[3:-1] - yVel[1:-3]) + yVel[:-4]) /(12*step)) 


    zDerivative = np.empty_like(zVel) 

    zDerivative[:2] = ((-zVel[2:4] + 4*zVel[1:3] - 3*zVel[:2]) / (2*step)) 
    zDerivative[-2:] = ((3*zVel[-2:] - 4*zVel[-3:-1] + zVel[-4:-2]) / (2*step)) 
    zDerivative[2:-2] = ((-zVel[4:] + 8*(zVel[3:-1] - zVel[1:-3]) + zVel[:-4]) /(12*step)) 

    allDerivatives = np.column_stack((xDerivative,yDerivative,zDerivative))

    return allDerivatives


In [ ]:
def calculateRMSE(observed, model):

    yObserved = observed
    yPredicted = model
    
    MSE = np.square(np.subtract(yObserved,yPredicted)).mean() 
    
    RMSE = math.sqrt(MSE)
    print("Root Mean Square Error:\n")
    print(RMSE)

    return RMSE

In [ ]:
def calculateModelVariances(modelSimulations):
    modelVariancesX = []
    for i in range(len(modelSimulations)):
        modelVariancesX.append(np.std(modelSimulations[i,:,0]))

    modelVariancesY = []
    for i in range(len(modelSimulations)):
        modelVariancesY.append(np.std(modelSimulations[i,:,1]))

    modelVariancesZ = []
    for i in range(len(modelSimulations)):
        modelVariancesZ.append(np.std(modelSimulations[i,:,2]))

    modelVariances = np.array((modelVariancesX,modelVariancesY,modelVariancesZ)).T

    return modelVariances

    

In [ ]:
def plotModelMeanALL(T, cond2GroupMean, cond3GroupMean, cond4GroupMean, cond5GroupMean, cond6GroupMean, cond7GroupMean, cond2GroupVariance, cond3GroupVariance, cond4GroupVariance, cond5GroupVariance, cond6GroupVariance, cond7GroupVariance, condName):
    fig = plt.figure(0)
    plt.title('Mean Trajectory')
   
    fig.text(0.5, 0.03, 'path length (\%)', ha='center', fontsize=Parameter['axes.labelsize'])
    fig.text(0.03, 0.5, 'hand movement (m)', va='center', rotation='vertical', fontsize=Parameter['axes.labelsize'])
    
    t = np.linspace(0,T)
    filteredTrajectory2 = signal.savgol_filter(cond2GroupMean, 7, 1, 0, axis=0)
    filteredTrajectory3 = signal.savgol_filter(cond3GroupMean, 7, 1, 0, axis=0)
    filteredTrajectory4 = signal.savgol_filter(cond4GroupMean, 7, 1, 0, axis=0)
    filteredTrajectory5 = signal.savgol_filter(cond5GroupMean, 7, 1, 0, axis=0)
    filteredTrajectory6 = signal.savgol_filter(cond6GroupMean, 7, 1, 0, axis=0)
    filteredTrajectory7 = signal.savgol_filter(cond7GroupMean, 7, 1, 0, axis=0)
    errorX2 = cond2GroupVariance[:,0]
    errorY2 = cond2GroupVariance[:,1]
    errorZ2 = cond2GroupVariance[:,2]
    errorX3 = cond3GroupVariance[:,0]
    errorY3 = cond3GroupVariance[:,1]
    errorZ3 = cond3GroupVariance[:,2]
    errorX4 = cond4GroupVariance[:,0]
    errorY4 = cond4GroupVariance[:,1]
    errorZ4 = cond4GroupVariance[:,2]
    errorX5 = cond5GroupVariance[:,0]
    errorY5 = cond5GroupVariance[:,1]
    errorZ5 = cond5GroupVariance[:,2]
    errorX6 = cond6GroupVariance[:,0]
    errorY6 = cond6GroupVariance[:,1]
    errorZ6 = cond6GroupVariance[:,2]
    errorX7 = cond7GroupVariance[:,0]
    errorY7 = cond7GroupVariance[:,1]
    errorZ7 = cond7GroupVariance[:,2]

    ax1 = plt.subplot2grid((6,3),(0,0))
    ax1.plot(t,filteredTrajectory2[:,0],linewidth=2, color='black', label='mean trajectory (m)')
    plt.fill_between(t,filteredTrajectory2[:,0] - errorX2, filteredTrajectory2[:,0]+errorX2,linewidth=2.0, color='lightgrey', label='standard deviation (m)')
    plt.xticks([])
    

    ax2 = plt.subplot2grid((6,3),(0,1))
    ax2.plot(t,filteredTrajectory2[:,1],linewidth=2, color='black')
    plt.fill_between(t,filteredTrajectory2[:,1] - errorY2, filteredTrajectory2[:,1]+errorY2,linewidth=2.0, color='lightgrey')

    plt.xticks([])
    
    ax3 = plt.subplot2grid((6,3),(0,2))
    ax3.plot(t,filteredTrajectory2[:,2],linewidth=2, color='black')
    plt.fill_between(t,filteredTrajectory2[:,2] - errorZ2, filteredTrajectory2[:,2]+errorZ2,linewidth=2.0, color='lightgrey')
  
    ax4 = plt.subplot2grid((6,3),(1,0), sharex = ax1)
    ax4.plot(t,filteredTrajectory3[:,0],linewidth=2, color='black')
    plt.fill_between(t,filteredTrajectory3[:,0] - errorX3, filteredTrajectory3[:,0]+errorX3,linewidth=2.0, color='lightgrey')

    ax5 = plt.subplot2grid((6,3),(1,1))
    ax5.plot(t,filteredTrajectory3[:,1],linewidth=2, color='black')
    plt.fill_between(t,filteredTrajectory3[:,1] - errorY3, filteredTrajectory3[:,1]+errorY3,linewidth=2.0, color='lightgrey')

    plt.xticks([])

    ax6 = plt.subplot2grid((6,3),(1,2))
    ax6.plot(t,filteredTrajectory3[:,2],linewidth=2, color='black')
    plt.fill_between(t,filteredTrajectory3[:,2] - errorZ3, filteredTrajectory3[:,2]+errorZ3,linewidth=2.0, color='lightgrey')
    
    ax7 = plt.subplot2grid((6,3),(2,0), sharex = ax1)
    ax7.plot(t,filteredTrajectory4[:,0],linewidth=2, color='black')
    plt.fill_between(t,filteredTrajectory4[:,0] - errorX4, filteredTrajectory4[:,0]+errorX4,linewidth=2.0, color='lightgrey')

    plt.xticks([])
    

    ax8 = plt.subplot2grid((6,3),(2,1))
    ax8.plot(t,filteredTrajectory4[:,1],linewidth=2, color='black')
    plt.fill_between(t,filteredTrajectory4[:,1] - errorY4, filteredTrajectory4[:,1]+errorY4,linewidth=2.0, color='lightgrey')

    plt.xticks([])

    ax9 = plt.subplot2grid((6,3),(2,2))
    ax9.plot(t,filteredTrajectory4[:,2],linewidth=2, color='black')
    plt.fill_between(t,filteredTrajectory4[:,2] - errorZ4, filteredTrajectory4[:,2]+errorZ4,linewidth=2.0, color='lightgrey')
    
    ax10 = plt.subplot2grid((6,3),(3,0), sharex = ax1)
    ax10.plot(t,filteredTrajectory5[:,0],linewidth=2, color='black')
    plt.fill_between(t,filteredTrajectory5[:,0] - errorX5, filteredTrajectory5[:,0]+errorX5,linewidth=2.0, color='lightgrey')
 
    plt.xticks([])
    

    ax11 = plt.subplot2grid((6,3),(3,1))
    ax11.plot(t,filteredTrajectory5[:,1],linewidth=2, color='black')
    plt.fill_between(t,filteredTrajectory5[:,1] - errorY5, filteredTrajectory5[:,1]+errorY5,linewidth=2.0, color='lightgrey')

    plt.xticks([])

    ax12 = plt.subplot2grid((6,3),(3,2))
    ax12.plot(t,filteredTrajectory5[:,2],linewidth=2, color='black')
    plt.fill_between(t,filteredTrajectory5[:,2] - errorZ5, filteredTrajectory5[:,2]+errorZ5,linewidth=2.0, color='lightgrey')

    ax13 = plt.subplot2grid((6,3),(4,0), sharex = ax1)
    ax13.plot(t,filteredTrajectory6[:,0],linewidth=2, color='black')
    plt.fill_between(t,filteredTrajectory6[:,0] - errorX6, filteredTrajectory6[:,0]+errorX6,linewidth=2.0, color='lightgrey')

    plt.xticks([])
    

    ax14 = plt.subplot2grid((6,3),(4,1))
    ax14.plot(t,filteredTrajectory6[:,1],linewidth=2, color='black')
    plt.fill_between(t,filteredTrajectory6[:,1] - errorY6, filteredTrajectory6[:,1]+errorY6,linewidth=2.0, color='lightgrey')

    plt.xticks([])

    ax15 = plt.subplot2grid((6,3),(4,2))
    ax15.plot(t,filteredTrajectory6[:,2],linewidth=2, color='black')
    plt.fill_between(t,filteredTrajectory6[:,2] - errorZ6, filteredTrajectory6[:,2]+errorZ6,linewidth=2.0, color='lightgrey')

    ax16 = plt.subplot2grid((6,3),(5,0), sharex = ax1)
    ax16.plot(t,filteredTrajectory7[:,0],linewidth=2, color='black')
    plt.fill_between(t,filteredTrajectory7[:,0] - errorX7, filteredTrajectory7[:,0]+errorX7,linewidth=2.0, color='lightgrey')

    plt.xticks([])
    

    ax17 = plt.subplot2grid((6,3),(5,1))
    ax17.plot(t,filteredTrajectory7[:,1],linewidth=2, color='black')
    plt.fill_between(t,filteredTrajectory7[:,1] - errorY7, filteredTrajectory7[:,1]+errorY7,linewidth=2.0, color='lightgrey')

    plt.xticks([])

    ax18 = plt.subplot2grid((6,3),(5,2))
    ax18.plot(t,filteredTrajectory7[:,2],linewidth=2, color='black')
    plt.fill_between(t,filteredTrajectory7[:,2] - errorZ7, filteredTrajectory7[:,2]+errorZ7,linewidth=2.0, color='lightgrey')
  
    fig.legend(loc='upper left',bbox_to_anchor=(0.08, 1.05))
    plt.savefig('.../' + condName + ".pdf")



In [ ]:
def plotModelMean(T, condModelMean, condModelVariance, condName): 

    t = np.linspace(0,T)
    #filteredTrajectory = signal.savgol_filter(condModelMean, 7, 1, 0, axis=0)
    errorX = condModelVariance[:,0]
    errorY = condModelVariance[:,1]
    errorZ = condModelVariance[:,2]
    plt.figure()
    plt.title('Mean Trajectory')


    plt.subplot(3, 1, 1)
    plt.plot(t,condModelMean[:,0],linewidth=2,label="Mean Trajectory", color='black')
    plt.fill_between(t,condModelMean[:,0] - errorX, condModelMean[:,0]+errorX,linewidth=2.0, color='lightgrey')

    plt.ylabel('forw. pos. (m)')
    plt.xticks([])
    plt.legend()

    plt.subplot(3, 1, 2)
    plt.plot(t,condModelMean[:,1],linewidth=2,label="Mean Trajectory", color='black')
    plt.fill_between(t,condModelMean[:,1] - errorY, condModelMean[:,1]+errorY,linewidth=2.0, color='lightgrey')
   
    plt.ylabel('lat. pos.')
    plt.xticks([])
   

    plt.subplot(3, 1, 3)
    plt.plot(t,condModelMean[:,2],linewidth=2,label="Mean Trajectory", color='black')
    plt.fill_between(t,condModelMean[:,2] - errorZ, condModelMean[:,2]+errorZ,linewidth=2.0, color='lightgrey')
    plt.xlabel('movement phase')
    plt.ylabel('vert. pos.')

    #plt.savefig('.../' + condName + ".pdf")



In [ ]:
def modelVariance(cond1ModelVariance):
    xVariances = cond1ModelVariance[:, :, 0]
    yVariances = cond1ModelVariance[:, :, 1]
    zVariances = cond1ModelVariance[:, :, 2]

    print(xVariances.shape)

    xGroupVariance = []
    for i in range(len(xVariances)):
        xGroupVariance.append(np.mean(xVariances[i,:]))

    zGroupVariance = []
    for i in range(len(zVariances)):
        zGroupVariance.append(np.mean(zVariances[i,:]))

    yGroupVariance = []
    for i in range(len(yVariances)):
        yGroupVariance.append(np.mean(yVariances[i,:]))

    GroupVariance = np.array((xGroupVariance,yGroupVariance,zGroupVariance)).T

    return GroupVariance
    

In [ ]:
def getInitialValues(condGroupMean, condGroupVel, condGroupVariance, nCondTrials):
    print('targetPosition is:', condGroupMean[-1,:])
    print('startvelocity is:', condGroupVel[0,:])
    print('initial uncertainty is:', condGroupVariance[0,:])

    print('timestep is:', dts[i])
    print('number of trials is:', nCondTrials)

    

In [ ]:
meanCond1P1 = np.asarray(pd.read_csv('...', header=None))
meanCond1P2 = np.asarray(pd.read_csv('...', header=None))
meanCond1P3 = np.asarray(pd.read_csv('...', header=None))
meanCond1P4 = np.asarray(pd.read_csv('...', header=None))
meanCond1P5 = np.asarray(pd.read_csv('...', header=None))
meanCond1P6 = np.asarray(pd.read_csv('...', header=None))
meanCond1P7 = np.asarray(pd.read_csv('...', header=None))
meanCond1P8 = np.asarray(pd.read_csv('...', header=None))
meanCond1P9 = np.asarray(pd.read_csv('...', header=None))
meanCond1P10 = np.asarray(pd.read_csv('...', header=None))

meanCond2LeftP1 = np.asarray(pd.read_csv('...', header=None))
meanCond2LeftP2 = np.asarray(pd.read_csv('...', header=None))
meanCond2LeftP3 = np.asarray(pd.read_csv('...', header=None))
meanCond2LeftP4 = np.asarray(pd.read_csv('...', header=None))
meanCond2LeftP5 = np.asarray(pd.read_csv('...', header=None))
meanCond2LeftP6 = np.asarray(pd.read_csv('...', header=None))
meanCond2LeftP7 = np.asarray(pd.read_csv('...', header=None))
meanCond2LeftP8 = np.asarray(pd.read_csv('...', header=None))
meanCond2LeftP9 = np.asarray(pd.read_csv('...', header=None))
meanCond2LeftP10 = np.asarray(pd.read_csv('...', header=None))


meanCond2RightP1 = np.asarray(pd.read_csv('...', header=None))
meanCond2RightP2 = np.asarray(pd.read_csv('...', header=None))
meanCond2RightP3 = np.asarray(pd.read_csv('...', header=None))
meanCond2RightP4 = np.asarray(pd.read_csv('...', header=None))
meanCond2RightP5 = np.asarray(pd.read_csv('...', header=None))
meanCond2RightP6 = np.asarray(pd.read_csv('...', header=None))
meanCond2RightP7 = np.asarray(pd.read_csv('...', header=None))
meanCond2RightP8 = np.asarray(pd.read_csv('...', header=None))
meanCond2RightP9 = np.asarray(pd.read_csv('...', header=None))
meanCond2RightP10 = np.asarray(pd.read_csv('...', header=None))


meanCond3LeftP1 = np.asarray(pd.read_csv('...', header=None))
meanCond3LeftP2 = np.asarray(pd.read_csv('...', header=None))
meanCond3LeftP3 = np.asarray(pd.read_csv('...', header=None))
meanCond3LeftP4 = np.asarray(pd.read_csv('...', header=None))
meanCond3LeftP5 = np.asarray(pd.read_csv('...', header=None))
meanCond3LeftP6 = np.asarray(pd.read_csv('...', header=None))
meanCond3LeftP7 = np.asarray(pd.read_csv('...', header=None))
meanCond3LeftP8 = np.asarray(pd.read_csv('...', header=None))
meanCond3LeftP9 = np.asarray(pd.read_csv('...', header=None))
meanCond3LeftP10 = np.asarray(pd.read_csv('...', header=None))


meanCond3RightP1 = np.asarray(pd.read_csv('...', header=None))
meanCond3RightP2 = np.asarray(pd.read_csv('...', header=None))
meanCond3RightP3 = np.asarray(pd.read_csv('...', header=None))
meanCond3RightP4 = np.asarray(pd.read_csv('...', header=None))
meanCond3RightP5 = np.asarray(pd.read_csv('...', header=None))
meanCond3RightP6 = np.asarray(pd.read_csv('...', header=None))
meanCond3RightP7 = np.asarray(pd.read_csv('...', header=None))
meanCond3RightP8 = np.asarray(pd.read_csv('...', header=None))
meanCond3RightP9 = np.asarray(pd.read_csv('...', header=None))
meanCond3RightP10 = np.asarray(pd.read_csv('...', header=None))


meanCond4LeftP1 = np.asarray(pd.read_csv('...', header=None))
meanCond4LeftP2 = np.asarray(pd.read_csv('...', header=None))
meanCond4LeftP3 = np.asarray(pd.read_csv('...', header=None))
meanCond4LeftP4 = np.asarray(pd.read_csv('...', header=None))
meanCond4LeftP5 = np.asarray(pd.read_csv('...', header=None))
meanCond4LeftP6 = np.asarray(pd.read_csv('...', header=None))
meanCond4LeftP7 = np.asarray(pd.read_csv('...', header=None))
meanCond4LeftP8 = np.asarray(pd.read_csv('...', header=None))
meanCond4LeftP9 = np.asarray(pd.read_csv('...', header=None))
meanCond4LeftP10 = np.asarray(pd.read_csv('...', header=None))


meanCond4RightP1 = np.asarray(pd.read_csv('...', header=None))
meanCond4RightP2 = np.asarray(pd.read_csv('...', header=None))
meanCond4RightP3 = np.asarray(pd.read_csv('...', header=None))
meanCond4RightP4 = np.asarray(pd.read_csv('...', header=None))
meanCond4RightP5 = np.asarray(pd.read_csv('...', header=None))
meanCond4RightP6 = np.asarray(pd.read_csv('...', header=None))
meanCond4RightP7 = np.asarray(pd.read_csv('...', header=None))
meanCond4RightP8 = np.asarray(pd.read_csv('...', header=None))
meanCond4RightP9 = np.asarray(pd.read_csv('...', header=None))
meanCond4RightP10 = np.asarray(pd.read_csv('...', header=None))


meanCond5LeftP1 = np.asarray(pd.read_csv('...', header=None))
meanCond5LeftP2 = np.asarray(pd.read_csv('...', header=None))
meanCond5LeftP3 = np.asarray(pd.read_csv('...', header=None))
meanCond5LeftP4 = np.asarray(pd.read_csv('...', header=None))
meanCond5LeftP5 = np.asarray(pd.read_csv('...', header=None))
meanCond5LeftP6 = np.asarray(pd.read_csv('...', header=None))
meanCond5LeftP7 = np.asarray(pd.read_csv('...', header=None))
meanCond5LeftP8 = np.asarray(pd.read_csv('...', header=None))
meanCond5LeftP9 = np.asarray(pd.read_csv('...', header=None))
meanCond5LeftP10 = np.asarray(pd.read_csv('...', header=None))


meanCond5RightP1 = np.asarray(pd.read_csv('...', header=None))
meanCond5RightP2 = np.asarray(pd.read_csv('...', header=None))
meanCond5RightP3 = np.asarray(pd.read_csv('...', header=None))
meanCond5RightP4 = np.asarray(pd.read_csv('...', header=None))
meanCond5RightP5 = np.asarray(pd.read_csv('...', header=None))
meanCond5RightP6 = np.asarray(pd.read_csv('...', header=None))
meanCond5RightP7 = np.asarray(pd.read_csv('...', header=None))
meanCond5RightP8 = np.asarray(pd.read_csv('...', header=None))
meanCond5RightP9 = np.asarray(pd.read_csv('...', header=None))
meanCond5RightP10 = np.asarray(pd.read_csv('...', header=None))


meanCond6LeftP1 = np.asarray(pd.read_csv('...', header=None))
meanCond6LeftP2 = np.asarray(pd.read_csv('...', header=None))
meanCond6LeftP3 = np.asarray(pd.read_csv('...', header=None))
meanCond6LeftP4 = np.asarray(pd.read_csv('...', header=None))
meanCond6LeftP5 = np.asarray(pd.read_csv('...', header=None))
meanCond6LeftP6 = np.asarray(pd.read_csv('...', header=None))
meanCond6LeftP7 = np.asarray(pd.read_csv('...', header=None))
meanCond6LeftP8 = np.asarray(pd.read_csv('...', header=None))
meanCond6LeftP9 = np.asarray(pd.read_csv('...', header=None))
meanCond6LeftP10 = np.asarray(pd.read_csv('...', header=None))


meanCond6RightP1 = np.asarray(pd.read_csv('...', header=None))
meanCond6RightP2 = np.asarray(pd.read_csv('...', header=None))
meanCond6RightP3 = np.asarray(pd.read_csv('...', header=None))
meanCond6RightP4 = np.asarray(pd.read_csv('...', header=None))
meanCond6RightP5 = np.asarray(pd.read_csv('...', header=None))
meanCond6RightP6 = np.asarray(pd.read_csv('...', header=None))
meanCond6RightP7 = np.asarray(pd.read_csv('...', header=None))
meanCond6RightP8 = np.asarray(pd.read_csv('...', header=None))
meanCond6RightP9 = np.asarray(pd.read_csv('...', header=None))
meanCond6RightP10 = np.asarray(pd.read_csv('...', header=None))


meanCond7LeftP1 = np.asarray(pd.read_csv('...', header=None))
meanCond7LeftP2 = np.asarray(pd.read_csv('...', header=None))
meanCond7LeftP3 = np.asarray(pd.read_csv('...', header=None))
meanCond7LeftP4 = np.asarray(pd.read_csv('...', header=None))
meanCond7LeftP5 = np.asarray(pd.read_csv('...', header=None))
meanCond7LeftP6 = np.asarray(pd.read_csv('...', header=None))
meanCond7LeftP7 = np.asarray(pd.read_csv('...', header=None))
meanCond7LeftP8 = np.asarray(pd.read_csv('...', header=None))
meanCond7LeftP9 = np.asarray(pd.read_csv('...', header=None))
meanCond7LeftP10 = np.asarray(pd.read_csv('...', header=None))


meanCond7RightP1 = np.asarray(pd.read_csv('...', header=None))
meanCond7RightP2 = np.asarray(pd.read_csv('...', header=None))
meanCond7RightP3 = np.asarray(pd.read_csv('...', header=None))
meanCond7RightP4 = np.asarray(pd.read_csv('...', header=None))
meanCond7RightP5 = np.asarray(pd.read_csv('...', header=None))
meanCond7RightP6 = np.asarray(pd.read_csv('...', header=None))
meanCond7RightP7 = np.asarray(pd.read_csv('...', header=None))
meanCond7RightP8 = np.asarray(pd.read_csv('...', header=None))
meanCond7RightP9 = np.asarray(pd.read_csv('...', header=None))
meanCond7RightP10 = np.asarray(pd.read_csv('...', header=None))

In [ ]:
meanCond1P1 = rescaleCoordinates1to5(meanCond1P1)
meanCond1P2 = rescaleCoordinates1to5(meanCond1P2)
meanCond1P3 = rescaleCoordinates1to5(meanCond1P3)
meanCond1P4 = rescaleCoordinates1to5(meanCond1P4)
meanCond1P5 = rescaleCoordinates1to5(meanCond1P5)

meanCond1P6 = rescaleCoordinates6to10(meanCond1P6)
meanCond1P7 = rescaleCoordinates6to10(meanCond1P7)
meanCond1P8 = rescaleCoordinates6to10(meanCond1P8)
meanCond1P9 = rescaleCoordinates6to10(meanCond1P9)
meanCond1P10 = rescaleCoordinates6to10(meanCond1P10)


meanCond2LeftP1 = rescaleCoordinates1to5(meanCond2LeftP1)
meanCond2LeftP2 = rescaleCoordinates1to5(meanCond2LeftP2)
meanCond2LeftP3 = rescaleCoordinates1to5(meanCond2LeftP3)
meanCond2LeftP4 = rescaleCoordinates1to5(meanCond2LeftP4)
meanCond2LeftP5 = rescaleCoordinates1to5(meanCond2LeftP5)

meanCond2LeftP6 = rescaleCoordinates6to10(meanCond2LeftP6)
meanCond2LeftP7 = rescaleCoordinates6to10(meanCond2LeftP7)
meanCond2LeftP8 = rescaleCoordinates6to10(meanCond2LeftP8)
meanCond2LeftP9 = rescaleCoordinates6to10(meanCond2LeftP9)
meanCond2LeftP10 = rescaleCoordinates6to10(meanCond2LeftP10)

meanCond2RightP1 = rescaleCoordinates1to5(meanCond2RightP1)
meanCond2RightP2 = rescaleCoordinates1to5(meanCond2RightP2)
meanCond2RightP3 = rescaleCoordinates1to5(meanCond2RightP3)
meanCond2RightP4 = rescaleCoordinates1to5(meanCond2RightP4)
meanCond2RightP5 = rescaleCoordinates1to5(meanCond2RightP5)

meanCond2RightP6 = rescaleCoordinates6to10(meanCond2RightP6)
meanCond2RightP7 = rescaleCoordinates6to10(meanCond2RightP7)
meanCond2RightP8 = rescaleCoordinates6to10(meanCond2RightP8)
meanCond2RightP9 = rescaleCoordinates6to10(meanCond2RightP9)
meanCond2RightP10 = rescaleCoordinates6to10(meanCond2RightP10)

meanCond3LeftP1 = rescaleCoordinates1to5(meanCond3LeftP1)
meanCond3LeftP2 = rescaleCoordinates1to5(meanCond3LeftP2)
meanCond3LeftP3 = rescaleCoordinates1to5(meanCond3LeftP3)
meanCond3LeftP4 = rescaleCoordinates1to5(meanCond3LeftP4)
meanCond3LeftP5 = rescaleCoordinates1to5(meanCond3LeftP5)

meanCond3LeftP6 = rescaleCoordinates6to10(meanCond3LeftP6)
meanCond3LeftP7 = rescaleCoordinates6to10(meanCond3LeftP7)
meanCond3LeftP8 = rescaleCoordinates6to10(meanCond3LeftP8)
meanCond3LeftP9 = rescaleCoordinates6to10(meanCond3LeftP9)
meanCond3LeftP10 = rescaleCoordinates6to10(meanCond3LeftP10)

meanCond3RightP1 = rescaleCoordinates1to5(meanCond3RightP1)
meanCond3RightP2 = rescaleCoordinates1to5(meanCond3RightP2)
meanCond3RightP3 = rescaleCoordinates1to5(meanCond3RightP3)
meanCond3RightP4 = rescaleCoordinates1to5(meanCond3RightP4)
meanCond3RightP5 = rescaleCoordinates1to5(meanCond3RightP5)

meanCond3RightP6 = rescaleCoordinates6to10(meanCond3RightP6)
meanCond3RightP7 = rescaleCoordinates6to10(meanCond3RightP7)
meanCond3RightP8 = rescaleCoordinates6to10(meanCond3RightP8)
meanCond3RightP9 = rescaleCoordinates6to10(meanCond3RightP9)
meanCond3RightP10 = rescaleCoordinates6to10(meanCond3RightP10)

meanCond4LeftP1 = rescaleCoordinates1to5(meanCond4LeftP1)
meanCond4LeftP2 = rescaleCoordinates1to5(meanCond4LeftP2)
meanCond4LeftP3 = rescaleCoordinates1to5(meanCond4LeftP3)
meanCond4LeftP4 = rescaleCoordinates1to5(meanCond4LeftP4)
meanCond4LeftP5 = rescaleCoordinates1to5(meanCond4LeftP5)

meanCond4LeftP6 = rescaleCoordinates6to10(meanCond4LeftP6)
meanCond4LeftP7 = rescaleCoordinates6to10(meanCond4LeftP7)
meanCond4LeftP8 = rescaleCoordinates6to10(meanCond4LeftP8)
meanCond4LeftP9 = rescaleCoordinates6to10(meanCond4LeftP9)
meanCond4LeftP10 = rescaleCoordinates6to10(meanCond4LeftP10)

meanCond4RightP1 = rescaleCoordinates1to5(meanCond4RightP1)
meanCond4RightP2 = rescaleCoordinates1to5(meanCond4RightP2)
meanCond4RightP3 = rescaleCoordinates1to5(meanCond4RightP3)
meanCond4RightP4 = rescaleCoordinates1to5(meanCond4RightP4)
meanCond4RightP5 = rescaleCoordinates1to5(meanCond4RightP5)

meanCond4RightP6 = rescaleCoordinates6to10(meanCond4RightP6)
meanCond4RightP7 = rescaleCoordinates6to10(meanCond4RightP7)
meanCond4RightP8 = rescaleCoordinates6to10(meanCond4RightP8)
meanCond4RightP9 = rescaleCoordinates6to10(meanCond4RightP9)
meanCond4RightP10 = rescaleCoordinates6to10(meanCond4RightP10)

meanCond5LeftP1 = rescaleCoordinates1to5(meanCond5LeftP1)
meanCond5LeftP2 = rescaleCoordinates1to5(meanCond5LeftP2)
meanCond5LeftP3 = rescaleCoordinates1to5(meanCond5LeftP3)
meanCond5LeftP4 = rescaleCoordinates1to5(meanCond5LeftP4)
meanCond5LeftP5 = rescaleCoordinates1to5(meanCond5LeftP5)

meanCond5LeftP6 = rescaleCoordinates6to10(meanCond5LeftP6)
meanCond5LeftP7 = rescaleCoordinates6to10(meanCond5LeftP7)
meanCond5LeftP8 = rescaleCoordinates6to10(meanCond5LeftP8)
meanCond5LeftP9 = rescaleCoordinates6to10(meanCond5LeftP9)
meanCond5LeftP10 = rescaleCoordinates6to10(meanCond5LeftP10)

meanCond5RightP1 = rescaleCoordinates1to5(meanCond5RightP1)
meanCond5RightP2 = rescaleCoordinates1to5(meanCond5RightP2)
meanCond5RightP3 = rescaleCoordinates1to5(meanCond5RightP3)
meanCond5RightP4 = rescaleCoordinates1to5(meanCond5RightP4)
meanCond5RightP5 = rescaleCoordinates1to5(meanCond5RightP5)

meanCond5RightP6 = rescaleCoordinates6to10(meanCond5RightP6)
meanCond5RightP7 = rescaleCoordinates6to10(meanCond5RightP7)
meanCond5RightP8 = rescaleCoordinates6to10(meanCond5RightP8)
meanCond5RightP9 = rescaleCoordinates6to10(meanCond5RightP9)
meanCond5RightP10 = rescaleCoordinates6to10(meanCond5RightP10)

meanCond6LeftP1 = rescaleCoordinates1to5(meanCond6LeftP1)
meanCond6LeftP2 = rescaleCoordinates1to5(meanCond6LeftP2)
meanCond6LeftP3 = rescaleCoordinates1to5(meanCond6LeftP3)
meanCond6LeftP4 = rescaleCoordinates1to5(meanCond6LeftP4)
meanCond6LeftP5 = rescaleCoordinates1to5(meanCond6LeftP5)

meanCond6LeftP6 = rescaleCoordinates6to10(meanCond6LeftP6)
meanCond6LeftP7 = rescaleCoordinates6to10(meanCond6LeftP7)
meanCond6LeftP8 = rescaleCoordinates6to10(meanCond6LeftP8)
meanCond6LeftP9 = rescaleCoordinates6to10(meanCond6LeftP9)
meanCond6LeftP10 = rescaleCoordinates6to10(meanCond6LeftP10)

meanCond6RightP1 = rescaleCoordinates1to5(meanCond6RightP1)
meanCond6RightP2 = rescaleCoordinates1to5(meanCond6RightP2)
meanCond6RightP3 = rescaleCoordinates1to5(meanCond6RightP3)
meanCond6RightP4 = rescaleCoordinates1to5(meanCond6RightP4)
meanCond6RightP5 = rescaleCoordinates1to5(meanCond6RightP5)

meanCond6RightP6 = rescaleCoordinates6to10(meanCond6RightP6)
meanCond6RightP7 = rescaleCoordinates6to10(meanCond6RightP7)
meanCond6RightP8 = rescaleCoordinates6to10(meanCond6RightP8)
meanCond6RightP9 = rescaleCoordinates6to10(meanCond6RightP9)
meanCond6RightP10 = rescaleCoordinates6to10(meanCond6RightP10)

meanCond7LeftP1 = rescaleCoordinates1to5(meanCond7LeftP1)
meanCond7LeftP2 = rescaleCoordinates1to5(meanCond7LeftP2)
meanCond7LeftP3 = rescaleCoordinates1to5(meanCond7LeftP3)
meanCond7LeftP4 = rescaleCoordinates1to5(meanCond7LeftP4)
meanCond7LeftP5 = rescaleCoordinates1to5(meanCond7LeftP5)

meanCond7LeftP6 = rescaleCoordinates6to10(meanCond7LeftP6)
meanCond7LeftP7 = rescaleCoordinates6to10(meanCond7LeftP7)
meanCond7LeftP8 = rescaleCoordinates6to10(meanCond7LeftP8)
meanCond7LeftP9 = rescaleCoordinates6to10(meanCond7LeftP9)
meanCond7LeftP10 = rescaleCoordinates6to10(meanCond7LeftP10)

meanCond7RightP1 = rescaleCoordinates1to5(meanCond7RightP1)
meanCond7RightP2 = rescaleCoordinates1to5(meanCond7RightP2)
meanCond7RightP3 = rescaleCoordinates1to5(meanCond7RightP3)
meanCond7RightP4 = rescaleCoordinates1to5(meanCond7RightP4)
meanCond7RightP5 = rescaleCoordinates1to5(meanCond7RightP5)

meanCond7RightP6 = rescaleCoordinates6to10(meanCond7RightP6)
meanCond7RightP7 = rescaleCoordinates6to10(meanCond7RightP7)
meanCond7RightP8 = rescaleCoordinates6to10(meanCond7RightP8)
meanCond7RightP9 = rescaleCoordinates6to10(meanCond7RightP9)
meanCond7RightP10 = rescaleCoordinates6to10(meanCond7RightP10)



In [ ]:
cond1GroupMean = groupMean(meanCond1P1, meanCond1P2, meanCond1P3, meanCond1P4, meanCond1P5, meanCond1P6, meanCond1P7, meanCond1P8, meanCond1P9, meanCond1P10)

cond2LeftGroupMean = groupMean(meanCond2LeftP1, meanCond2LeftP2, meanCond2LeftP3, meanCond2LeftP4, meanCond2LeftP5, meanCond2LeftP6, meanCond2LeftP7, meanCond2LeftP8, meanCond2LeftP9, meanCond2LeftP10)
cond2RightGroupMean = groupMean(meanCond2RightP1, meanCond2RightP2, meanCond2RightP3, meanCond2RightP4, meanCond2RightP5, meanCond2RightP6, meanCond2RightP7, meanCond2RightP8, meanCond2RightP9, meanCond2RightP10)

cond3LeftGroupMean = groupMean(meanCond3LeftP1, meanCond3LeftP2, meanCond3LeftP3, meanCond3LeftP4, meanCond3LeftP5, meanCond3LeftP6, meanCond3LeftP7, meanCond3LeftP8, meanCond3LeftP9, meanCond3LeftP10)
cond3RightGroupMean = groupMean(meanCond3RightP1, meanCond3RightP2, meanCond3RightP3, meanCond3RightP4, meanCond3RightP5, meanCond3RightP6, meanCond3RightP7, meanCond3RightP8, meanCond3RightP9, meanCond3RightP10)

cond4LeftGroupMean = groupMean(meanCond4LeftP1, meanCond4LeftP2, meanCond4LeftP3, meanCond4LeftP4, meanCond4LeftP5, meanCond4LeftP6, meanCond4LeftP7, meanCond4LeftP8, meanCond4LeftP9, meanCond4LeftP10)
cond4RightGroupMean = groupMean(meanCond4RightP1, meanCond4RightP2, meanCond4RightP3, meanCond4RightP4, meanCond4RightP5, meanCond4RightP6, meanCond4RightP7, meanCond4RightP8, meanCond4RightP9, meanCond4RightP10)

cond5LeftGroupMean = groupMean(meanCond5LeftP1, meanCond5LeftP2, meanCond5LeftP3, meanCond5LeftP4, meanCond5LeftP5, meanCond5LeftP6, meanCond5LeftP7, meanCond5LeftP8, meanCond5LeftP9, meanCond5LeftP10)
cond5RightGroupMean = groupMean(meanCond5RightP1, meanCond5RightP2, meanCond5RightP3, meanCond5RightP4, meanCond5RightP5, meanCond5RightP6, meanCond5RightP7, meanCond5RightP8, meanCond5RightP9, meanCond5RightP10)

cond6LeftGroupMean = groupMean(meanCond6LeftP1, meanCond6LeftP2, meanCond6LeftP3, meanCond6LeftP4, meanCond6LeftP5, meanCond6LeftP6, meanCond6LeftP7, meanCond6LeftP8, meanCond6LeftP9, meanCond6LeftP10)
cond6RightGroupMean = groupMean(meanCond6RightP1, meanCond6RightP2, meanCond6RightP3, meanCond6RightP4, meanCond6RightP5, meanCond6RightP6, meanCond6RightP7, meanCond6RightP8, meanCond6RightP9, meanCond6RightP10)

cond7LeftGroupMean = groupMean(meanCond7LeftP1, meanCond7LeftP2, meanCond7LeftP3, meanCond7LeftP4, meanCond7LeftP5, meanCond7LeftP6, meanCond7LeftP7, meanCond7LeftP8, meanCond7LeftP9, meanCond7LeftP10)
cond7RightGroupMean = groupMean(meanCond7RightP1, meanCond7RightP2, meanCond7RightP3, meanCond7RightP4, meanCond7RightP5, meanCond7RightP6, meanCond7RightP7, meanCond7RightP8, meanCond7RightP9, meanCond7RightP10)

##########################  Variances Analysis #########################

In [ ]:
varCond1P1 = np.asarray(pd.read_csv('...', header=None))
varCond1P2 = np.asarray(pd.read_csv('...', header=None))
varCond1P3 = np.asarray(pd.read_csv('...', header=None))
varCond1P4 = np.asarray(pd.read_csv('...', header=None))
varCond1P5 = np.asarray(pd.read_csv('...', header=None))
varCond1P6 = np.asarray(pd.read_csv('...', header=None))
varCond1P7 = np.asarray(pd.read_csv('...', header=None))
varCond1P8 = np.asarray(pd.read_csv('...', header=None))
varCond1P9 = np.asarray(pd.read_csv('...', header=None))
varCond1P10 = np.asarray(pd.read_csv('...', header=None))


varCond2LeftP1 = np.asarray(pd.read_csv('...', header=None))
varCond2LeftP2 = np.asarray(pd.read_csv('...', header=None))
varCond2LeftP3 = np.asarray(pd.read_csv('...', header=None))
varCond2LeftP4 = np.asarray(pd.read_csv('...', header=None))
varCond2LeftP5 = np.asarray(pd.read_csv('...', header=None))
varCond2LeftP6 = np.asarray(pd.read_csv('...', header=None))
varCond2LeftP7 = np.asarray(pd.read_csv('...', header=None))
varCond2LeftP8 = np.asarray(pd.read_csv('...', header=None))
varCond2LeftP9 = np.asarray(pd.read_csv('...', header=None))
varCond2LeftP10 = np.asarray(pd.read_csv('...', header=None))


varCond2RightP1 = np.asarray(pd.read_csv('...', header=None))
varCond2RightP2 = np.asarray(pd.read_csv('...', header=None))
varCond2RightP3 = np.asarray(pd.read_csv('...', header=None))
varCond2RightP4 = np.asarray(pd.read_csv('...', header=None))
varCond2RightP5 = np.asarray(pd.read_csv('...', header=None))
varCond2RightP6 = np.asarray(pd.read_csv('...', header=None))
varCond2RightP7 = np.asarray(pd.read_csv('...', header=None))
varCond2RightP8 = np.asarray(pd.read_csv('...', header=None))
varCond2RightP9 = np.asarray(pd.read_csv('...', header=None))
varCond2RightP10 = np.asarray(pd.read_csv('...', header=None))


varCond3LeftP1 = np.asarray(pd.read_csv('...', header=None))
varCond3LeftP2 = np.asarray(pd.read_csv('...', header=None))
varCond3LeftP3 = np.asarray(pd.read_csv('...', header=None))
varCond3LeftP4 = np.asarray(pd.read_csv('...', header=None))
varCond3LeftP5 = np.asarray(pd.read_csv('...', header=None))
varCond3LeftP6 = np.asarray(pd.read_csv('...', header=None))
varCond3LeftP7 = np.asarray(pd.read_csv('...', header=None))
varCond3LeftP8 = np.asarray(pd.read_csv('...', header=None))
varCond3LeftP9 = np.asarray(pd.read_csv('...', header=None))
varCond3LeftP10 = np.asarray(pd.read_csv('...', header=None))


varCond3RightP1 = np.asarray(pd.read_csv('...', header=None))
varCond3RightP2 = np.asarray(pd.read_csv('...', header=None))
varCond3RightP3 = np.asarray(pd.read_csv('...', header=None))
varCond3RightP4 = np.asarray(pd.read_csv('...', header=None))
varCond3RightP5 = np.asarray(pd.read_csv('...', header=None))
varCond3RightP6 = np.asarray(pd.read_csv('...', header=None))
varCond3RightP7 = np.asarray(pd.read_csv('...', header=None))
varCond3RightP8 = np.asarray(pd.read_csv('...', header=None))
varCond3RightP9 = np.asarray(pd.read_csv('...', header=None))
varCond3RightP10 = np.asarray(pd.read_csv('...', header=None))


varCond4LeftP1 = np.asarray(pd.read_csv('...', header=None))
varCond4LeftP2 = np.asarray(pd.read_csv('...', header=None))
varCond4LeftP3 = np.asarray(pd.read_csv('...', header=None))
varCond4LeftP4 = np.asarray(pd.read_csv('...', header=None))
varCond4LeftP5 = np.asarray(pd.read_csv('...', header=None))
varCond4LeftP6 = np.asarray(pd.read_csv('...', header=None))
varCond4LeftP7 = np.asarray(pd.read_csv('...', header=None))
varCond4LeftP8 = np.asarray(pd.read_csv('...', header=None))
varCond4LeftP9 = np.asarray(pd.read_csv('...', header=None))
varCond4LeftP10 = np.asarray(pd.read_csv('...', header=None))


varCond4RightP1 = np.asarray(pd.read_csv('...', header=None))
varCond4RightP2 = np.asarray(pd.read_csv('...', header=None))
varCond4RightP3 = np.asarray(pd.read_csv('...', header=None))
varCond4RightP4 = np.asarray(pd.read_csv('...', header=None))
varCond4RightP5 = np.asarray(pd.read_csv('...', header=None))
varCond4RightP6 = np.asarray(pd.read_csv('...', header=None))
varCond4RightP7 = np.asarray(pd.read_csv('...', header=None))
varCond4RightP8 = np.asarray(pd.read_csv('...', header=None))
varCond4RightP9 = np.asarray(pd.read_csv('...', header=None))
varCond4RightP10 = np.asarray(pd.read_csv('...', header=None))


varCond5LeftP1 = np.asarray(pd.read_csv('...', header=None))
varCond5LeftP2 = np.asarray(pd.read_csv('...', header=None))
varCond5LeftP3 = np.asarray(pd.read_csv('...', header=None))
varCond5LeftP4 = np.asarray(pd.read_csv('...', header=None))
varCond5LeftP5 = np.asarray(pd.read_csv('...', header=None))
varCond5LeftP6 = np.asarray(pd.read_csv('...', header=None))
varCond5LeftP7 = np.asarray(pd.read_csv('...', header=None))
varCond5LeftP8 = np.asarray(pd.read_csv('...', header=None))
varCond5LeftP9 = np.asarray(pd.read_csv('...', header=None))
varCond5LeftP10 = np.asarray(pd.read_csv('...', header=None))


varCond5RightP1 = np.asarray(pd.read_csv('...', header=None))
varCond5RightP2 = np.asarray(pd.read_csv('...', header=None))
varCond5RightP3 = np.asarray(pd.read_csv('...', header=None))
varCond5RightP4 = np.asarray(pd.read_csv('...', header=None))
varCond5RightP5 = np.asarray(pd.read_csv('...', header=None))
varCond5RightP6 = np.asarray(pd.read_csv('...', header=None))
varCond5RightP7 = np.asarray(pd.read_csv('...', header=None))
varCond5RightP8 = np.asarray(pd.read_csv('...', header=None))
varCond5RightP9 = np.asarray(pd.read_csv('...', header=None))
varCond5RightP10 = np.asarray(pd.read_csv('...', header=None))


varCond6LeftP1 = np.asarray(pd.read_csv('...', header=None))
varCond6LeftP2 = np.asarray(pd.read_csv('...', header=None))
varCond6LeftP3 = np.asarray(pd.read_csv('...', header=None))
varCond6LeftP4 = np.asarray(pd.read_csv('...', header=None))
varCond6LeftP5 = np.asarray(pd.read_csv('...', header=None))
varCond6LeftP6 = np.asarray(pd.read_csv('...', header=None))
varCond6LeftP7 = np.asarray(pd.read_csv('...', header=None))
varCond6LeftP8 = np.asarray(pd.read_csv('...', header=None))
varCond6LeftP9 = np.asarray(pd.read_csv('...', header=None))
varCond6LeftP10 = np.asarray(pd.read_csv('...', header=None))


varCond6RightP1 = np.asarray(pd.read_csv('...', header=None))
varCond6RightP2 = np.asarray(pd.read_csv('...', header=None))
varCond6RightP3 = np.asarray(pd.read_csv('...', header=None))
varCond6RightP4 = np.asarray(pd.read_csv('...', header=None))
varCond6RightP5 = np.asarray(pd.read_csv('...', header=None))
varCond6RightP6 = np.asarray(pd.read_csv('...', header=None))
varCond6RightP7 = np.asarray(pd.read_csv('...', header=None))
varCond6RightP8 = np.asarray(pd.read_csv('...', header=None))
varCond6RightP9 = np.asarray(pd.read_csv('...', header=None))
varCond6RightP10 = np.asarray(pd.read_csv('...', header=None))


varCond7LeftP1 = np.asarray(pd.read_csv('...', header=None))
varCond7LeftP2 = np.asarray(pd.read_csv('...', header=None))
varCond7LeftP3 = np.asarray(pd.read_csv('...', header=None))
varCond7LeftP4 = np.asarray(pd.read_csv('...', header=None))
varCond7LeftP5 = np.asarray(pd.read_csv('...', header=None))
varCond7LeftP6 = np.asarray(pd.read_csv('...', header=None))
varCond7LeftP7 = np.asarray(pd.read_csv('...', header=None))
varCond7LeftP8 = np.asarray(pd.read_csv('...', header=None))
varCond7LeftP9 = np.asarray(pd.read_csv('...', header=None))
varCond7LeftP10 = np.asarray(pd.read_csv('...', header=None))


varCond7RightP1 = np.asarray(pd.read_csv('...', header=None))
varCond7RightP2 = np.asarray(pd.read_csv('...', header=None))
varCond7RightP3 = np.asarray(pd.read_csv('...', header=None))
varCond7RightP4 = np.asarray(pd.read_csv('...', header=None))
varCond7RightP5 = np.asarray(pd.read_csv('...', header=None))
varCond7RightP6 = np.asarray(pd.read_csv('...', header=None))
varCond7RightP7 = np.asarray(pd.read_csv('...', header=None))
varCond7RightP8 = np.asarray(pd.read_csv('...', header=None))
varCond7RightP9 = np.asarray(pd.read_csv('...', header=None))
varCond7RightP10 = np.asarray(pd.read_csv('...', header=None))

In [ ]:
cond1GroupVariance = groupVariance(varCond1P1, varCond1P2, varCond1P3, varCond1P4, varCond1P5, varCond1P6, varCond1P7, varCond1P8, varCond1P9, varCond1P10)

cond2LeftGroupVariance = groupVariance(varCond2LeftP1, varCond2LeftP2, varCond2LeftP3, varCond2LeftP4, varCond2LeftP5, varCond2LeftP6, varCond2LeftP7, varCond2LeftP8, varCond2LeftP9, varCond2LeftP10)
cond2RightGroupVariance = groupVariance(varCond2RightP1, varCond2RightP2, varCond2RightP3, varCond2RightP4, varCond2RightP5, varCond2RightP6, varCond2RightP7, varCond2RightP8, varCond2RightP9, varCond2RightP10)

cond3LeftGroupVariance = groupVariance(varCond3LeftP1, varCond3LeftP2, varCond3LeftP3, varCond3LeftP4, varCond3LeftP5, varCond3LeftP6, varCond3LeftP7, varCond3LeftP8, varCond3LeftP9, varCond3LeftP10)
cond3RightGroupVariance = groupVariance(varCond3RightP1, varCond3RightP2, varCond3RightP3, varCond3RightP4, varCond3RightP5, varCond3RightP6, varCond3RightP7, varCond3RightP8, varCond3RightP9, varCond3RightP10)

cond4LeftGroupVariance = groupVariance(varCond4LeftP1, varCond4LeftP2, varCond4LeftP3, varCond4LeftP4, varCond4LeftP5, varCond4LeftP6, varCond4LeftP7, varCond4LeftP8, varCond4LeftP9, varCond4LeftP10)
cond4RightGroupVariance = groupVariance(varCond4RightP1, varCond4RightP2, varCond4RightP3, varCond4RightP4, varCond4RightP5, varCond4RightP6, varCond4RightP7, varCond4RightP8, varCond4RightP9, varCond4RightP10)

cond5LeftGroupVariance = groupVariance(varCond5LeftP1, varCond5LeftP2, varCond5LeftP3, varCond5LeftP4, varCond5LeftP5, varCond5LeftP6, varCond5LeftP7, varCond5LeftP8, varCond5LeftP9, varCond5LeftP10)
cond5RightGroupVariance = groupVariance(varCond5RightP1, varCond5RightP2, varCond5RightP3, varCond5RightP4, varCond5RightP5, varCond5RightP6, varCond5RightP7, varCond5RightP8, varCond5RightP9, varCond5RightP10)

cond6LeftGroupVariance = groupVariance(varCond6LeftP1, varCond6LeftP2, varCond6LeftP3, varCond6LeftP4, varCond6LeftP5, varCond6LeftP6, varCond6LeftP7, varCond6LeftP8, varCond6LeftP9, varCond6LeftP10)
cond6RightGroupVariance = groupVariance(varCond6RightP1, varCond6RightP2, varCond6RightP3, varCond6RightP4, varCond6RightP5, varCond6RightP6, varCond6RightP7, varCond6RightP8, varCond6RightP9, varCond6RightP10)

cond7LeftGroupVariance = groupVariance(varCond7LeftP1, varCond7LeftP2, varCond7LeftP3, varCond7LeftP4, varCond7LeftP5, varCond7LeftP6, varCond7LeftP7, varCond7LeftP8, varCond7LeftP9, varCond7LeftP10)
cond7RightGroupVariance = groupVariance(varCond7RightP1, varCond7RightP2, varCond7RightP3, varCond7RightP4, varCond7RightP5, varCond7RightP6, varCond7RightP7, varCond7RightP8, varCond7RightP9, varCond7RightP10)

In [ ]:
varVelCond1P1 = np.asarray(pd.read_csv('...', header=None))
varVelCond1P2 = np.asarray(pd.read_csv('...', header=None))
varVelCond1P3 = np.asarray(pd.read_csv('...', header=None))
varVelCond1P4 = np.asarray(pd.read_csv('...', header=None))
varVelCond1P5 = np.asarray(pd.read_csv('...', header=None))
varVelCond1P6 = np.asarray(pd.read_csv('...', header=None))
varVelCond1P7 = np.asarray(pd.read_csv('...', header=None))
varVelCond1P8 = np.asarray(pd.read_csv('...', header=None))
varVelCond1P9 = np.asarray(pd.read_csv('...', header=None))
varVelCond1P10 = np.asarray(pd.read_csv('...', header=None))


varVelCond2LeftP1 = np.asarray(pd.read_csv('...', header=None))
varVelCond2LeftP2 = np.asarray(pd.read_csv('...', header=None))
varVelCond2LeftP3 = np.asarray(pd.read_csv('...', header=None))
varVelCond2LeftP4 = np.asarray(pd.read_csv('...', header=None))
varVelCond2LeftP5 = np.asarray(pd.read_csv('...', header=None))
varVelCond2LeftP6 = np.asarray(pd.read_csv('...', header=None))
varVelCond2LeftP7 = np.asarray(pd.read_csv('...', header=None))
varVelCond2LeftP8 = np.asarray(pd.read_csv('...', header=None))
varVelCond2LeftP9 = np.asarray(pd.read_csv('...', header=None))
varVelCond2LeftP10 = np.asarray(pd.read_csv('...', header=None))


varVelCond2RightP1 = np.asarray(pd.read_csv('...', header=None))
varVelCond2RightP2 = np.asarray(pd.read_csv('...', header=None))
varVelCond2RightP3 = np.asarray(pd.read_csv('...', header=None))
varVelCond2RightP4 = np.asarray(pd.read_csv('...', header=None))
varVelCond2RightP5 = np.asarray(pd.read_csv('...', header=None))
varVelCond2RightP6 = np.asarray(pd.read_csv('...', header=None))
varVelCond2RightP7 = np.asarray(pd.read_csv('...', header=None))
varVelCond2RightP8 = np.asarray(pd.read_csv('...', header=None))
varVelCond2RightP9 = np.asarray(pd.read_csv('...', header=None))
varVelCond2RightP10 = np.asarray(pd.read_csv('...', header=None))


varVelCond3LeftP1 = np.asarray(pd.read_csv('...', header=None))
varVelCond3LeftP2 = np.asarray(pd.read_csv('...', header=None))
varVelCond3LeftP3 = np.asarray(pd.read_csv('...', header=None))
varVelCond3LeftP4 = np.asarray(pd.read_csv('...', header=None))
varVelCond3LeftP5 = np.asarray(pd.read_csv('...', header=None))
varVelCond3LeftP6 = np.asarray(pd.read_csv('...', header=None))
varVelCond3LeftP7 = np.asarray(pd.read_csv('...', header=None))
varVelCond3LeftP8 = np.asarray(pd.read_csv('...', header=None))
varVelCond3LeftP9 = np.asarray(pd.read_csv('...', header=None))
varVelCond3LeftP10 = np.asarray(pd.read_csv('...', header=None))


varVelCond3RightP1 = np.asarray(pd.read_csv('...', header=None))
varVelCond3RightP2 = np.asarray(pd.read_csv('...', header=None))
varVelCond3RightP3 = np.asarray(pd.read_csv('...', header=None))
varVelCond3RightP4 = np.asarray(pd.read_csv('...', header=None))
varVelCond3RightP5 = np.asarray(pd.read_csv('...', header=None))
varVelCond3RightP6 = np.asarray(pd.read_csv('...', header=None))
varVelCond3RightP7 = np.asarray(pd.read_csv('...', header=None))
varVelCond3RightP8 = np.asarray(pd.read_csv('...', header=None))
varVelCond3RightP9 = np.asarray(pd.read_csv('...', header=None))
varVelCond3RightP10 = np.asarray(pd.read_csv('...', header=None))


varVelCond4LeftP1 = np.asarray(pd.read_csv('...', header=None))
varVelCond4LeftP2 = np.asarray(pd.read_csv('...', header=None))
varVelCond4LeftP3 = np.asarray(pd.read_csv('...', header=None))
varVelCond4LeftP4 = np.asarray(pd.read_csv('...', header=None))
varVelCond4LeftP5 = np.asarray(pd.read_csv('...', header=None))
varVelCond4LeftP6 = np.asarray(pd.read_csv('...', header=None))
varVelCond4LeftP7 = np.asarray(pd.read_csv('...', header=None))
varVelCond4LeftP8 = np.asarray(pd.read_csv('...', header=None))
varVelCond4LeftP9 = np.asarray(pd.read_csv('...', header=None))
varVelCond4LeftP10 = np.asarray(pd.read_csv('...', header=None))


varVelCond4RightP1 = np.asarray(pd.read_csv('...', header=None))
varVelCond4RightP2 = np.asarray(pd.read_csv('...', header=None))
varVelCond4RightP3 = np.asarray(pd.read_csv('...', header=None))
varVelCond4RightP4 = np.asarray(pd.read_csv('...', header=None))
varVelCond4RightP5 = np.asarray(pd.read_csv('...', header=None))
varVelCond4RightP6 = np.asarray(pd.read_csv('...', header=None))
varVelCond4RightP7 = np.asarray(pd.read_csv('...', header=None))
varVelCond4RightP8 = np.asarray(pd.read_csv('...', header=None))
varVelCond4RightP9 = np.asarray(pd.read_csv('...', header=None))
varVelCond4RightP10 = np.asarray(pd.read_csv('...', header=None))


varVelCond5LeftP1 = np.asarray(pd.read_csv('...', header=None))
varVelCond5LeftP2 = np.asarray(pd.read_csv('...', header=None))
varVelCond5LeftP3 = np.asarray(pd.read_csv('...', header=None))
varVelCond5LeftP4 = np.asarray(pd.read_csv('...', header=None))
varVelCond5LeftP5 = np.asarray(pd.read_csv('...', header=None))
varVelCond5LeftP6 = np.asarray(pd.read_csv('...', header=None))
varVelCond5LeftP7 = np.asarray(pd.read_csv('...', header=None))
varVelCond5LeftP8 = np.asarray(pd.read_csv('...', header=None))
varVelCond5LeftP9 = np.asarray(pd.read_csv('...', header=None))
varVelCond5LeftP10 = np.asarray(pd.read_csv('...', header=None))


varVelCond5RightP1 = np.asarray(pd.read_csv('...', header=None))
varVelCond5RightP2 = np.asarray(pd.read_csv('...', header=None))
varVelCond5RightP3 = np.asarray(pd.read_csv('...', header=None))
varVelCond5RightP4 = np.asarray(pd.read_csv('...', header=None))
varVelCond5RightP5 = np.asarray(pd.read_csv('...', header=None))
varVelCond5RightP6 = np.asarray(pd.read_csv('...', header=None))
varVelCond5RightP7 = np.asarray(pd.read_csv('...', header=None))
varVelCond5RightP8 = np.asarray(pd.read_csv('...', header=None))
varVelCond5RightP9 = np.asarray(pd.read_csv('...', header=None))
varVelCond5RightP10 = np.asarray(pd.read_csv('...', header=None))


varVelCond6LeftP1 = np.asarray(pd.read_csv('...', header=None))
varVelCond6LeftP2 = np.asarray(pd.read_csv('...', header=None))
varVelCond6LeftP3 = np.asarray(pd.read_csv('...', header=None))
varVelCond6LeftP4 = np.asarray(pd.read_csv('...', header=None))
varVelCond6LeftP5 = np.asarray(pd.read_csv('...', header=None))
varVelCond6LeftP6 = np.asarray(pd.read_csv('...', header=None))
varVelCond6LeftP7 = np.asarray(pd.read_csv('...', header=None))
varVelCond6LeftP8 = np.asarray(pd.read_csv('...', header=None))
varVelCond6LeftP9 = np.asarray(pd.read_csv('...', header=None))
varVelCond6LeftP10 = np.asarray(pd.read_csv('...', header=None))


varVelCond6RightP1 = np.asarray(pd.read_csv('...', header=None))
varVelCond6RightP2 = np.asarray(pd.read_csv('...', header=None))
varVelCond6RightP3 = np.asarray(pd.read_csv('...', header=None))
varVelCond6RightP4 = np.asarray(pd.read_csv('...', header=None))
varVelCond6RightP5 = np.asarray(pd.read_csv('...', header=None))
varVelCond6RightP6 = np.asarray(pd.read_csv('...', header=None))
varVelCond6RightP7 = np.asarray(pd.read_csv('...', header=None))
varVelCond6RightP8 = np.asarray(pd.read_csv('...', header=None))
varVelCond6RightP9 = np.asarray(pd.read_csv('...', header=None))
varVelCond6RightP10 = np.asarray(pd.read_csv('...', header=None))


varVelCond7LeftP1 = np.asarray(pd.read_csv('...', header=None))
varVelCond7LeftP2 = np.asarray(pd.read_csv('...', header=None))
varVelCond7LeftP3 = np.asarray(pd.read_csv('...', header=None))
varVelCond7LeftP4 = np.asarray(pd.read_csv('...', header=None))
varVelCond7LeftP5 = np.asarray(pd.read_csv('...', header=None))
varVelCond7LeftP6 = np.asarray(pd.read_csv('...', header=None))
varVelCond7LeftP7 = np.asarray(pd.read_csv('...', header=None))
varVelCond7LeftP8 = np.asarray(pd.read_csv('...', header=None))
varVelCond7LeftP9 = np.asarray(pd.read_csv('...', header=None))
varVelCond7LeftP10 = np.asarray(pd.read_csv('...', header=None))


varVelCond7RightP1 = np.asarray(pd.read_csv('...', header=None))
varVelCond7RightP2 = np.asarray(pd.read_csv('...', header=None))
varVelCond7RightP3 = np.asarray(pd.read_csv('...', header=None))
varVelCond7RightP4 = np.asarray(pd.read_csv('...', header=None))
varVelCond7RightP5 = np.asarray(pd.read_csv('...', header=None))
varVelCond7RightP6 = np.asarray(pd.read_csv('...', header=None))
varVelCond7RightP7 = np.asarray(pd.read_csv('...', header=None))
varVelCond7RightP8 = np.asarray(pd.read_csv('...', header=None))
varVelCond7RightP9 = np.asarray(pd.read_csv('...', header=None))
varVelCond7RightP10 = np.asarray(pd.read_csv('...', header=None))

In [ ]:
cond1GroupVarianceVel = groupVariance(varVelCond1P1, varVelCond1P2, varVelCond1P3, varVelCond1P4, varVelCond1P5, varVelCond1P6, varVelCond1P7, varVelCond1P8, varVelCond1P9, varVelCond1P10)

cond2LeftGroupVarianceVel = groupVariance(varVelCond2LeftP1, varVelCond2LeftP2, varVelCond2LeftP3, varVelCond2LeftP4, varVelCond2LeftP5, varVelCond2LeftP6, varVelCond2LeftP7, varVelCond2LeftP8, varVelCond2LeftP9, varVelCond2LeftP10)
cond2RightGroupVarianceVel = groupVariance(varVelCond2RightP1, varVelCond2RightP2, varVelCond2RightP3, varVelCond2RightP4, varVelCond2RightP5, varVelCond2RightP6, varVelCond2RightP7, varVelCond2RightP8, varVelCond2RightP9, varVelCond2RightP10)

cond3LeftGroupVarianceVel = groupVariance(varVelCond3LeftP1, varVelCond3LeftP2, varVelCond3LeftP3, varVelCond3LeftP4, varVelCond3LeftP5, varVelCond3LeftP6, varVelCond3LeftP7, varVelCond3LeftP8, varVelCond3LeftP9, varVelCond3LeftP10)
cond3RightGroupVarianceVel = groupVariance(varVelCond3RightP1, varVelCond3RightP2, varVelCond3RightP3, varVelCond3RightP4, varVelCond3RightP5, varVelCond3RightP6, varVelCond3RightP7, varVelCond3RightP8, varVelCond3RightP9, varVelCond3RightP10)

cond4LeftGroupVarianceVel = groupVariance(varVelCond4LeftP1, varVelCond4LeftP2, varVelCond4LeftP3, varVelCond4LeftP4, varVelCond4LeftP5, varVelCond4LeftP6, varVelCond4LeftP7, varVelCond4LeftP8, varVelCond4LeftP9, varVelCond4LeftP10)
cond4RightGroupVarianceVel = groupVariance(varVelCond4RightP1, varVelCond4RightP2, varVelCond4RightP3, varVelCond4RightP4, varVelCond4RightP5, varVelCond4RightP6, varVelCond4RightP7, varVelCond4RightP8, varVelCond4RightP9, varVelCond4RightP10)

cond5LeftGroupVarianceVel = groupVariance(varVelCond5LeftP1, varVelCond5LeftP2, varVelCond5LeftP3, varVelCond5LeftP4, varVelCond5LeftP5, varVelCond5LeftP6, varVelCond5LeftP7, varVelCond5LeftP8, varVelCond5LeftP9, varVelCond5LeftP10)
cond5RightGroupVarianceVel = groupVariance(varVelCond5RightP1, varVelCond5RightP2, varVelCond5RightP3, varVelCond5RightP4, varVelCond5RightP5, varVelCond5RightP6, varVelCond5RightP7, varVelCond5RightP8, varVelCond5RightP9, varVelCond5RightP10)

cond6LeftGroupVarianceVel = groupVariance(varVelCond6LeftP1, varVelCond6LeftP2, varVelCond6LeftP3, varVelCond6LeftP4, varVelCond6LeftP5, varVelCond6LeftP6, varVelCond6LeftP7, varVelCond6LeftP8, varVelCond6LeftP9, varVelCond6LeftP10)
cond6RightGroupVarianceVel = groupVariance(varVelCond6RightP1, varVelCond6RightP2, varVelCond6RightP3, varVelCond6RightP4, varVelCond6RightP5, varVelCond6RightP6, varVelCond6RightP7, varVelCond6RightP8, varVelCond6RightP9, varVelCond6RightP10)

cond7LeftGroupVarianceVel = groupVariance(varVelCond7LeftP1, varVelCond7LeftP2, varVelCond7LeftP3, varVelCond7LeftP4, varVelCond7LeftP5, varVelCond7LeftP6, varVelCond7LeftP7, varVelCond7LeftP8, varVelCond7LeftP9, varVelCond7LeftP10)
cond7RightGroupVarianceVel = groupVariance(varVelCond7RightP1, varVelCond7RightP2, varVelCond7RightP3, varVelCond7RightP4, varVelCond7RightP5, varVelCond7RightP6, varVelCond7RightP7, varVelCond7RightP8, varVelCond7RightP9, varVelCond7RightP10)

In [ ]:
dts = []
PertSteps = []
for i in range(len(GroupMeanPertOnsets)):
    dts.append(GroupMeanMoveDurations[i+1]/50)
    PertSteps.append(GroupMeanPertOnsets[i]/dts[i])

print(dts)
print(PertSteps)

##########################   Perturbation Onsets ########################   (deprecated)

In [ ]:
""" pertOnsetsP1 = np.asarray(pd.read_csv('...', header=None))
pertOnsetsP2 = np.asarray(pd.read_csv('...', header=None))
pertOnsetsP3 = np.asarray(pd.read_csv('...', header=None))
pertOnsetsP4 = np.asarray(pd.read_csv('...', header=None))
pertOnsetsP5 = np.asarray(pd.read_csv('...', header=None))
pertOnsetsP6 = np.asarray(pd.read_csv('...', header=None))
pertOnsetsP7 = np.asarray(pd.read_csv('...', header=None))
pertOnsetsP8 = np.asarray(pd.read_csv('...', header=None))
pertOnsetsP9 = np.asarray(pd.read_csv('...', header=None))
pertOnsetsP10 = np.asarray(pd.read_csv('...', header=None)) """

In [ ]:
#GroupMeanPertOnsets = GroupMeanPertOnset(pertOnsetsP1, pertOnsetsP2, pertOnsetsP3, pertOnsetsP4, pertOnsetsP5, pertOnsetsP6, pertOnsetsP7, pertOnsetsP8, pertOnsetsP9, pertOnsetsP10)

##################### Mean Move Duration ##############################

In [ ]:
MoveDurationsP1 = np.asarray(pd.read_csv('...', header=None))
MoveDurationsP2 = np.asarray(pd.read_csv('...', header=None))
MoveDurationsP3 = np.asarray(pd.read_csv('...', header=None))
MoveDurationsP4 = np.asarray(pd.read_csv('...', header=None))
MoveDurationsP5 = np.asarray(pd.read_csv('...', header=None))
MoveDurationsP6 = np.asarray(pd.read_csv('...', header=None))
MoveDurationsP7 = np.asarray(pd.read_csv('...', header=None))
MoveDurationsP8 = np.asarray(pd.read_csv('...', header=None))
MoveDurationsP9 = np.asarray(pd.read_csv('...', header=None))
MoveDurationsP10 = np.asarray(pd.read_csv('...', header=None))

In [ ]:
GroupMeanMoveDurations = GroupMeanMoveDuration(MoveDurationsP1, MoveDurationsP2, MoveDurationsP3, MoveDurationsP4, MoveDurationsP5, MoveDurationsP6, MoveDurationsP7, MoveDurationsP8, MoveDurationsP9, MoveDurationsP10)

##################### Plot Group Means ############################

In [ ]:
FinalMean = TotalMean(cond1GroupMean, cond2LeftGroupMean, cond2RightGroupMean,cond3LeftGroupMean, cond3RightGroupMean,cond4LeftGroupMean, cond4RightGroupMean,cond5LeftGroupMean, cond5RightGroupMean,cond6LeftGroupMean, cond6RightGroupMean,cond7LeftGroupMean, cond7RightGroupMean )

In [ ]:
FinalVel = TotalMean(cond1Vel, cond2LeftVel, cond2RightVel,cond3LeftVel, cond3RightVel,cond4LeftVel, cond4RightVel,cond5LeftVel, cond5RightVel,cond6LeftVel, cond6RightVel,cond7LeftVel, cond7RightVel )

In [ ]:
plotGroupMeanCond1(1, cond2LeftGroupMean, cond3LeftGroupMean, cond4LeftGroupMean, cond5LeftGroupMean, cond6LeftGroupMean, cond7LeftGroupMean, cond2LeftGroupVariance, cond3LeftGroupVariance, cond4LeftGroupVariance, cond5LeftGroupVariance, cond6LeftGroupVariance, cond7LeftGroupVariance, 'FinalMeanLeftPlot')

In [ ]:
plotGroupMeanCond1(1, cond2RightGroupMean, cond3RightGroupMean, cond4RightGroupMean, cond5RightGroupMean, cond6RightGroupMean, cond7RightGroupMean, cond2RightGroupVariance, cond3RightGroupVariance, cond4RightGroupVariance, cond5RightGroupVariance, cond6RightGroupVariance, cond7RightGroupVariance, 'FinalMeanRightPlot')

In [ ]:
plotGroupMeanCond1(1, cond1GroupMean, cond1GroupVariance, 'cond1GroupMeanTrajectory')

#########################  Plot Group Velocities #############################

In [ ]:
cond1Vel = finiteDifferenceHigherOrder(cond1GroupMean[:,0], cond1GroupMean[:,1], cond1GroupMean[:,2])

cond2LeftVel = finiteDifferenceHigherOrder(cond2LeftGroupMean[:,0], cond2LeftGroupMean[:,1], cond2LeftGroupMean[:,2])
cond2RightVel = finiteDifferenceHigherOrder(cond2RightGroupMean[:,0], cond2RightGroupMean[:,1], cond2RightGroupMean[:,2])

cond3LeftVel = finiteDifferenceHigherOrder(cond3LeftGroupMean[:,0], cond3LeftGroupMean[:,1], cond3LeftGroupMean[:,2])
cond3RightVel = finiteDifferenceHigherOrder(cond3RightGroupMean[:,0], cond3RightGroupMean[:,1], cond3RightGroupMean[:,2])

cond4LeftVel = finiteDifferenceHigherOrder(cond4LeftGroupMean[:,0], cond4LeftGroupMean[:,1], cond4LeftGroupMean[:,2])
cond4RightVel = finiteDifferenceHigherOrder(cond4RightGroupMean[:,0], cond4RightGroupMean[:,1], cond4RightGroupMean[:,2])

cond5RightVel = finiteDifferenceHigherOrder(cond5RightGroupMean[:,0], cond5RightGroupMean[:,1], cond5RightGroupMean[:,2])
cond5LeftVel = finiteDifferenceHigherOrder(cond5LeftGroupMean[:,0], cond5LeftGroupMean[:,1], cond5LeftGroupMean[:,2])

cond6LeftVel = finiteDifferenceHigherOrder(cond6LeftGroupMean[:,0], cond6LeftGroupMean[:,1], cond6LeftGroupMean[:,2])
cond6RightVel = finiteDifferenceHigherOrder(cond6RightGroupMean[:,0], cond6RightGroupMean[:,1], cond6RightGroupMean[:,2])

cond7LeftVel = finiteDifferenceHigherOrder(cond7LeftGroupMean[:,0], cond7LeftGroupMean[:,1], cond7LeftGroupMean[:,2])
cond7RightVel = finiteDifferenceHigherOrder(cond7RightGroupMean[:,0], cond7RightGroupMean[:,1], cond7RightGroupMean[:,2])

In [ ]:
plotGroupMeanVel(GroupMeanMoveDurations[0], cond1Vel, 'cond1GroupMeanVelocity')

plotGroupMeanVel(GroupMeanMoveDurations[1], cond2LeftVel, 'cond2LeftGroupMeanVelocity')
plotGroupMeanVel(GroupMeanMoveDurations[2], cond2RightVel, 'cond2RightGroupMeanVelocity')

plotGroupMeanVel(GroupMeanMoveDurations[3], cond3LeftVel, 'cond3LeftGroupMeanVelocity')
plotGroupMeanVel(GroupMeanMoveDurations[4], cond3RightVel, 'cond3RightGroupMeanVelocity')

plotGroupMeanVel(GroupMeanMoveDurations[5], cond4LeftVel, 'cond4LeftGroupMeanVelocity')
plotGroupMeanVel(GroupMeanMoveDurations[6], cond4RightVel, 'cond4RightGroupMeanVelocity')

plotGroupMeanVel(GroupMeanMoveDurations[7], cond5LeftVel, 'cond5LeftGroupMeanVelocity')
plotGroupMeanVel(GroupMeanMoveDurations[8], cond5RightVel, 'cond5RightGroupMeanVelocity')

plotGroupMeanVel(GroupMeanMoveDurations[9], cond6LeftVel, 'cond6LeftGroupMeanVelocity')
plotGroupMeanVel(GroupMeanMoveDurations[10], cond6RightVel, 'cond6RightGroupMeanVelocity')

plotGroupMeanVel(GroupMeanMoveDurations[11], cond7LeftVel, 'cond7LeftGroupMeanVelocity')
plotGroupMeanVel(GroupMeanMoveDurations[12], cond7RightVel, 'cond7RightGroupMeanVelocity')

In [ ]:
#auch die vels nochmal timeless:
plotGroupMeanVel(1, cond1Vel, 'cond1GroupMeanVelocity')# 0)

plotGroupMeanVel(1, cond2LeftVel, 'cond2LeftGroupMeanVelocity')
plotGroupMeanVel(1, cond2RightVel, 'cond2RightGroupMeanVelocity')

plotGroupMeanVel(1, cond3LeftVel, 'cond3LeftGroupMeanVelocity')
plotGroupMeanVel(1, cond3RightVel, 'cond3RightGroupMeanVelocity')

plotGroupMeanVel(1, cond4LeftVel, 'cond4LeftGroupMeanVelocity')
plotGroupMeanVel(1, cond4RightVel, 'cond4RightGroupMeanVelocity')

plotGroupMeanVel(1, cond5LeftVel, 'cond5LeftGroupMeanVelocity')
plotGroupMeanVel(1, cond5RightVel, 'cond5RightGroupMeanVelocity')

plotGroupMeanVel(1, cond6LeftVel, 'cond6LeftGroupMeanVelocity')
plotGroupMeanVel(1, cond6RightVel, 'cond6RightGroupMeanVelocity')

plotGroupMeanVel(1, cond7LeftVel, 'cond7LeftGroupMeanVelocity')
plotGroupMeanVel(1, cond7RightVel, 'cond7RightGroupMeanVelocity')

In [ ]:
plotGroupMeanALL(1, cond2LeftVel, cond3LeftVel, cond4LeftVel, cond5LeftVel, cond6LeftVel, cond7LeftVel, cond2LeftGroupVarianceVel, cond3LeftGroupVarianceVel, cond4LeftGroupVarianceVel, cond5LeftGroupVarianceVel, cond6LeftGroupVarianceVel, cond7LeftGroupVarianceVel, 'FinalVelLeftPlot')

###############  Group Variance Development ###################

In [ ]:
plt.style.use(['high-vis'])
fig, ax = plt.subplots(nrows=1, ncols=2, sharex=True, sharey=True, figsize=(16, 6))
fig.text(0.5, 0.0, 'path length (\%)', ha='center', fontsize=Parameter['axes.labelsize'])
fig.text(0.06, 0.5, 'standard deviation (m)', va='center', rotation='vertical', fontsize=Parameter['axes.labelsize'])

ax1 = plt.subplot(121)
plt.plot(np.linspace(0,1), signal.savgol_filter(cond2LeftGroupVariance[:,1], 7, 1, 0), label='PD 12.5cm')
plt.plot(np.linspace(0,1), signal.savgol_filter(cond3LeftGroupVariance[:,1], 7, 1, 0), label='PD 15.0cm')
plt.plot(np.linspace(0,1), signal.savgol_filter(cond4LeftGroupVariance[:,1], 7, 1, 0), label='PD 17.5cm')
plt.plot(np.linspace(0,1), signal.savgol_filter(cond5LeftGroupVariance[:,1], 7, 1, 0), label='PD 20.0cm')
plt.plot(np.linspace(0,1), signal.savgol_filter(cond6LeftGroupVariance[:,1], 7, 1, 0), label='PD 22.5cm')
plt.plot(np.linspace(0,1), signal.savgol_filter(cond7LeftGroupVariance[:,1], 7, 1, 0), label='PD 25.0cm')

ax2 = plt.subplot(122)
plt.plot(np.linspace(0,1), signal.savgol_filter(cond2RightGroupVariance[:,1], 7, 1, 0))
plt.plot(np.linspace(0,1), signal.savgol_filter(cond3RightGroupVariance[:,1], 7, 1, 0))
plt.plot(np.linspace(0,1), signal.savgol_filter(cond4RightGroupVariance[:,1], 7, 1, 0))
plt.plot(np.linspace(0,1), signal.savgol_filter(cond5RightGroupVariance[:,1], 7, 1, 0))
plt.plot(np.linspace(0,1), signal.savgol_filter(cond6RightGroupVariance[:,1], 7, 1, 0))
plt.plot(np.linspace(0,1), signal.savgol_filter(cond7RightGroupVariance[:,1], 7, 1, 0))
fig.legend(loc='lower center', bbox_to_anchor=(0.2, 0.45),prop={'size': 18})

plt.savefig('...')

In [ ]:
plotModelVelALL(1, cond2RightModelVelFinal, cond3RightModelVelFinal, cond4RightModelVelFinal, cond5RightModelVelFinal, cond6RightModelVelFinal, cond7RightModelVelFinal, cond2RightModelVelVariancesFinal, cond3RightModelVelVariancesFinal, cond4RightModelVelVariancesFinal, cond5RightModelVelVariancesFinal, cond6RightModelVelVariancesFinal, cond7RightModelVelVariancesFinal, 'ModelVelsRightFinal')

In [ ]:
plotModelVelALL(1, cond2LeftModelVel, cond3LeftModelVel, cond4LeftModelVel, cond5LeftModelVel, cond6LeftModelVel, cond7LeftModelVel, 'ModelVelsLeftFinal')

In [ ]:
cond1ModelVariancesFinal = calculateModelVariances(cond1ModelSimulationsFinal)

cond2LeftModelVariancesFinal = calculateModelVariances(cond2LeftModelSimulationsFinal)
cond2RightModelVariancesFinal = calculateModelVariances(cond2RightModelSimulationsFinal)

cond3LeftModelVariancesFinal = calculateModelVariances(cond3LeftModelSimulationsFinal)
cond3RightModelVariancesFinal = calculateModelVariances(cond3RightModelSimulationsFinal)

cond4LeftModelVariancesFinal = calculateModelVariances(cond4LeftModelSimulationsFinal)
cond4RightModelVariancesFinal = calculateModelVariances(cond4RightModelSimulationsFinal)

cond5LeftModelVariancesFinal = calculateModelVariances(cond5LeftModelSimulationsFinal)
cond5RightModelVariancesFinal = calculateModelVariances(cond5RightModelSimulationsFinal)

cond6LeftModelVariancesFinal = calculateModelVariances(cond6LeftModelSimulationsFinal)
cond6RightModelVariancesFinal = calculateModelVariances(cond6RightModelSimulationsFinal)

cond7LeftModelVariancesFinal = calculateModelVariances(cond7LeftModelSimulationsFinal)
cond7RightModelVariancesFinal = calculateModelVariances(cond7RightModelSimulationsFinal)

########################## Get Values for Modelling #################################

In [ ]:
#getInitialValues(cond2LeftGroupMean, cond2LeftVel, cond2LeftGroupVariance)

##############################  Model Means and Vels ###################################

In [ ]:
cond1ModelMeanFinal = np.load('...', allow_pickle=True)

cond2LeftModelMeanFinal = np.load('...', allow_pickle=True)
cond2RightModelMeanFinal = np.load('...', allow_pickle=True)

cond3LeftModelMeanFinal = np.load('...', allow_pickle=True)
cond3RightModelMeanFinal = np.load('...', allow_pickle=True)

cond4LeftModelMeanFinal = np.load('...', allow_pickle=True)
cond4RightModelMeanFinal = np.load('...', allow_pickle=True)

cond5LeftModelMeanFinal = np.load('...', allow_pickle=True)
cond5RightModelMeanFinal = np.load('...', allow_pickle=True)

cond6LeftModelMeanFinal = np.load('...', allow_pickle=True)
cond6RightModelMeanFinal = np.load('...', allow_pickle=True)

cond7LeftModelMeanFinal = np.load('...', allow_pickle=True)
cond7RightModelMeanFinal = np.load('...', allow_pickle=True)


In [ ]:
cond1ModelSimulationsFinal = np.load('...', allow_pickle=True)

cond2LeftModelSimulationsFinal = np.load('...', allow_pickle=True)
cond2RightModelSimulationsFinal = np.load('...', allow_pickle=True)

cond3LeftModelSimulationsFinal = np.load('...', allow_pickle=True)
cond3RightModelSimulationsFinal = np.load('...', allow_pickle=True)

cond4LeftModelSimulationsFinal = np.load('...', allow_pickle=True)
cond4RightModelSimulationsFinal = np.load('...', allow_pickle=True)

cond5LeftModelSimulationsFinal = np.load('...', allow_pickle=True)
cond5RightModelSimulationsFinal = np.load('...', allow_pickle=True)

cond6LeftModelSimulationsFinal = np.load('...', allow_pickle=True)
cond6RightModelSimulationsFinal = np.load('...', allow_pickle=True)

cond7LeftModelSimulationsFinal = np.load('...', allow_pickle=True)
cond7RightModelSimulationsFinal = np.load('...', allow_pickle=True) 

In [ ]:
cond1ModelVelFinal = np.load('...', allow_pickle=True)

cond2LeftModelVelFinal = np.load('...', allow_pickle=True)
cond2RightModelVelFinal = np.load('...', allow_pickle=True)

cond3LeftModelVelFinal = np.load('...', allow_pickle=True)
cond3RightModelVelFinal = np.load('...', allow_pickle=True)

cond4LeftModelVelFinal = np.load('...', allow_pickle=True)
cond4RightModelVelFinal = np.load('...', allow_pickle=True)

cond5LeftModelVelFinal = np.load('...', allow_pickle=True)
cond5RightModelVelFinal = np.load('...', allow_pickle=True)

cond6LeftModelVelFinal = np.load('...', allow_pickle=True)
cond6RightModelVelFinal = np.load('...', allow_pickle=True)

cond7LeftModelVelFinal = np.load('...', allow_pickle=True)
cond7RightModelVelFinal = np.load('...', allow_pickle=True)


In [ ]:
cond1VelModelSimulationsFinal = np.load('...', allow_pickle=True)

cond2LeftVelModelSimulationsFinal = np.load('...', allow_pickle=True)
cond2RightVelModelSimulationsFinal = np.load('...', allow_pickle=True)

cond3LeftVelModelSimulationsFinal = np.load('...', allow_pickle=True)
cond3RightVelModelSimulationsFinal = np.load('...', allow_pickle=True)

cond4LeftVelModelSimulationsFinal = np.load('...', allow_pickle=True)
cond4RightVelModelSimulationsFinal = np.load('...', allow_pickle=True)

cond5LeftVelModelSimulationsFinal = np.load('...', allow_pickle=True)
cond5RightVelModelSimulationsFinal = np.load('...', allow_pickle=True)

cond6LeftVelModelSimulationsFinal = np.load('...', allow_pickle=True)
cond6RightVelModelSimulationsFinal = np.load('...', allow_pickle=True)

cond7LeftVelModelSimulationsFinal = np.load('...', allow_pickle=True)
cond7RightVelModelSimulationsFinal = np.load('...', allow_pickle=True) 

##############################  Model Variances ##################################

In [ ]:
cond1ModelVelVariancesFinal = calculateModelVariances(cond1VelModelSimulationsFinal)

cond2LeftModelVelVariancesFinal = calculateModelVariances(cond2LeftVelModelSimulationsFinal)
cond2RightModelVelVariancesFinal = calculateModelVariances(cond2RightVelModelSimulationsFinal)

cond3LeftModelVelVariancesFinal = calculateModelVariances(cond3LeftVelModelSimulationsFinal)
cond3RightModelVelVariancesFinal = calculateModelVariances(cond3RightVelModelSimulationsFinal)

cond4LeftModelVelVariancesFinal = calculateModelVariances(cond4LeftVelModelSimulationsFinal)
cond4RightModelVelVariancesFinal = calculateModelVariances(cond4RightVelModelSimulationsFinal)

cond5LeftModelVelVariancesFinal = calculateModelVariances(cond5LeftVelModelSimulationsFinal)
cond5RightModelVelVariancesFinal = calculateModelVariances(cond5RightVelModelSimulationsFinal)

cond6LeftModelVelVariancesFinal = calculateModelVariances(cond6LeftVelModelSimulationsFinal)
cond6RightModelVelVariancesFinal = calculateModelVariances(cond6RightVelModelSimulationsFinal)

cond7LeftModelVelVariancesFinal = calculateModelVariances(cond7LeftVelModelSimulationsFinal)
cond7RightModelVelVariancesFinal = calculateModelVariances(cond7RightVelModelSimulationsFinal)

######################  Make Model Means + get RMSE ####################################

In [ ]:
def modelPrediction(condGroupMean, condGroupVariance, condModelMean, condModelSimulations, name):
    
    condModelVariances = calculateModelVariances(condModelSimulations)
    condObservedDistribution = np.column_stack([condGroupMean, condGroupVariance])
    condPredictedDistribution = np.column_stack([condModelMean, condModelVariances])

    plotModelMean(1, condModelMean, condModelVariances, name)

    RMSE = calculateRMSE(condObservedDistribution, condPredictedDistribution)

    return RMSE

In [ ]:
cond1RMSE = modelPrediction(cond1GroupMean, cond1GroupVariance, cond1ModelMeanFinal, cond1ModelSimulationsFinal, 'cond1Prediction')

cond2LeftRMSE = modelPrediction(cond2LeftGroupMean, cond2LeftGroupVariance, cond2LeftModelMeanFinal, cond2LeftModelSimulationsFinal, 'cond2LeftPrediction')
cond2RightRMSE = modelPrediction(cond2RightGroupMean, cond2RightGroupVariance, cond2RightModelMeanFinal, cond2RightModelSimulationsFinal, 'cond2RightPrediction')

cond3LeftRMSE = modelPrediction(cond3LeftGroupMean, cond3LeftGroupVariance, cond3LeftModelMeanFinal, cond3LeftModelSimulationsFinal, 'cond3LeftPrediction')
cond3RightRMSE = modelPrediction(cond3RightGroupMean, cond3RightGroupVariance, cond3RightModelMeanFinal, cond3RightModelSimulationsFinal, 'cond3RightPrediction')

cond4LeftRMSE = modelPrediction(cond4LeftGroupMean, cond4LeftGroupVariance, cond4LeftModelMeanFinal, cond4LeftModelSimulationsFinal, 'cond4LeftPrediction')
cond4RightRMSE = modelPrediction(cond4RightGroupMean, cond4RightGroupVariance, cond4RightModelMeanFinal, cond4RightModelSimulationsFinal, 'cond4RightPrediction')

cond5LeftRMSE = modelPrediction(cond5LeftGroupMean, cond5LeftGroupVariance, cond5LeftModelMeanFinal, cond5LeftModelSimulationsFinal, 'cond5LeftPrediction')
cond5RightRMSE = modelPrediction(cond5RightGroupMean, cond5RightGroupVariance, cond5RightModelMeanFinal, cond5RightModelSimulationsFinal, 'cond5RightPrediction')

cond6LeftRMSE = modelPrediction(cond6LeftGroupMean, cond6LeftGroupVariance, cond6LeftModelMeanFinal, cond6LeftModelSimulationsFinal, 'cond6LeftPrediction')
cond6RightRMSE = modelPrediction(cond6RightGroupMean, cond6RightGroupVariance, cond6RightModelMeanFinal, cond6RightModelSimulationsFinal, 'cond6RightPrediction')

cond7LeftRMSE = modelPrediction(cond7LeftGroupMean, cond7LeftGroupVariance, cond7LeftModelMeanFinal, cond7LeftModelSimulationsFinal, 'cond7LeftPrediction')
cond7RightRMSE = modelPrediction(cond7RightGroupMean, cond7RightGroupVariance, cond7RightModelMeanFinal, cond7RightModelSimulationsFinal, 'cond7RightPrediction')


In [ ]:
AllRMSEsLeft = [cond1RMSE, cond2LeftRMSE, cond3LeftRMSE, cond4LeftRMSE, cond5LeftRMSE, cond6LeftRMSE, cond7LeftRMSE]
AllRMSEsRight = [cond1RMSE, cond2RightRMSE, cond3RightRMSE, cond4RightRMSE, cond5RightRMSE, cond6RightRMSE, cond7RightRMSE]

In [ ]:
AllRMSEsLeft = roundList(AllRMSEsLeft)
AllRMSEsRight = roundList(AllRMSEsRight)

In [ ]:
""" import plotly.graph_objects as go

fig = go.Figure(data=[go.Table(header=dict(values=['Perturbation Distance (cm)', 'RMSE Perturbation Left', 'RMSE Perturbation Right']),
                 cells=dict(values=[['No Perturbation', 12.5, 15.0, 17.5, 20.0, 22.5, 25.0], AllRMSEsLeft, AllRMSEsRight]))
                     ])
fig.show()
fig.write_image("...")

"""

In [ ]:
cond1ObservedDistribution = np.column_stack([cond1GroupMean, cond1GroupVariance])

In [ ]:
cond2LeftModelVariancesTest = calculateModelVariances(cond2LeftModelVariancesTest)

In [ ]:
cond1PredictedDistribution = np.column_stack([cond1ModelMeanFinal, cond1ModelVariancesFinal])

In [ ]:
cond2LeftPredictedDistribution = np.column_stack([cond2LeftModelMeanTest, cond2LeftModelVariancesTest])

In [ ]:
cond2LeftObservedDistribution = np.column_stack([cond2LeftGroupMean, cond2LeftGroupVariance])

In [ ]:
RMSE = calculateRMSE(cond2LeftObservedDistribution, cond2LeftPredictedDistribution)

In [ ]:
""" import plotly.graph_objects as go

fig = go.Figure(data=[go.Table(header=dict(values=['Perturbation Distance (cm)', 'r', 'v', 'f', 'j', 'RMSE']),
                 cells=dict(values=[['No Perturbation', '12.5 Left', '12.5 Right', '15.0 Left', '15.0 Right', '17.5 Left', '17.5 Right', '20.0 Left', '20.0 Right', '22.5 Left', '22.5 Right', '25.0 Left', '25.0 Right'], [rLeft[0], rLeft[1], rRight[1], rLeft[2], rRight[2], rLeft[3], rRight[3], rLeft[4], rRight[4], rLeft[5], rRight[5], rLeft[6], rRight[6]], [vLeft[0], vLeft[1], vRight[1], vLeft[2], vRight[2], vLeft[3], vRight[3], vLeft[4], vRight[4], vLeft[5], vRight[5], vLeft[6], vRight[6]], [fLeft[0], fLeft[1], fRight[1], fLeft[2], fRight[2], fLeft[3], fRight[3], fLeft[4], fRight[4], fLeft[5], fRight[5], fLeft[6], fRight[6]], [jLeft[0], jLeft[1], jRight[1], jLeft[2], jRight[2], jLeft[3], jRight[3], jLeft[4], jRight[4], jLeft[5], jRight[5], jLeft[6], jRight[6]], [AllRMSEsLeft[0], AllRMSEsLeft[1], AllRMSEsRight[1], AllRMSEsLeft[2], AllRMSEsRight[2], AllRMSEsLeft[3], AllRMSEsRight[3], AllRMSEsLeft[4], AllRMSEsRight[4], AllRMSEsLeft[5], AllRMSEsRight[5], AllRMSEsLeft[6], AllRMSEsRight[6]]], font=dict( size = 12), height=20))
                     ])

fig.show()
fig.write_image("...")

 """

In [ ]:
AllRMSEsLeft[0], AllRMSEsLeft[1], AllRMSEsRight[1], AllRMSEsLeft[2], AllRMSEsRight[2], AllRMSEsLeft[3], AllRMSEsRight[3], AllRMSEsLeft[4], AllRMSEsRight[4], AllRMSEsLeft[5], AllRMSEsRight[5], AllRMSEsLeft[6], AllRMSEsRight[6]

In [ ]:
plt.style.use(['high-vis'])
fig, ax = plt.subplots(nrows=1, ncols=2, sharex=True, sharey=True, figsize=(16, 6))
fig.text(0.5, 0.0, 'path length (\%)', ha='center', fontsize=Parameter['axes.labelsize'])
fig.text(0.05, 0.5, 'standard deviation (m)', va='center', rotation='vertical', fontsize=Parameter['axes.labelsize'])

ax1 = plt.subplot(121)
plt.plot(np.linspace(0,1), cond2LeftModelVariancesFinal[:,1], label='PD1')
plt.plot(np.linspace(0,1), cond3LeftModelVariancesFinal[:,1], label='PD2')
plt.plot(np.linspace(0,1), cond4LeftModelVariancesFinal[:,1], label='PD3')
plt.plot(np.linspace(0,1), cond5LeftModelVariancesFinal[:,1], label='PD4')
plt.plot(np.linspace(0,1), cond6LeftModelVariancesFinal[:,1], label='PD5')
plt.plot(np.linspace(0,1), cond7LeftModelVariancesFinal[:,1], label='PD6')

ax2 = plt.subplot(122)
plt.plot(np.linspace(0,1), cond2RightModelVariancesFinal[:,1])
plt.plot(np.linspace(0,1), cond3RightModelVariancesFinal[:,1])
plt.plot(np.linspace(0,1), cond4RightModelVariancesFinal[:,1])
plt.plot(np.linspace(0,1), cond5RightModelVariancesFinal[:,1])
plt.plot(np.linspace(0,1), cond6RightModelVariancesFinal[:,1])
plt.plot(np.linspace(0,1), cond7RightModelVariancesFinal[:,1])
fig.legend(loc='lower center', bbox_to_anchor=(0.511, 0.54),prop={'size': 16})
#plt.savefig('...')

####################### Variance Development Model ##############################

In [ ]:
def unpickleSuccesses(infile = '.../SuccessTable.pkl'):

    infile = open(infile,'rb')
    Successes = pickle.load(infile)
    infile.close()

    return Successes#mean movedurations:
P1Successes = unpickleSuccesses(infile = '/SuccessTable.pkl')
P2Successes = unpickleSuccesses(infile = '/SuccessTable.pkl')
P3Successes = unpickleSuccesses(infile = '/SuccessTable.pkl')
P4Successes = unpickleSuccesses(infile = '/SuccessTable.pkl')
P5Successes = unpickleSuccesses(infile = '/SuccessTable.pkl')
P6Successes = unpickleSuccesses(infile = '/SuccessTable.pkl')
P7Successes = unpickleSuccesses(infile = '/SuccessTable.pkl')
P8Successes = unpickleSuccesses(infile = '/SuccessTable.pkl')
P9Successes = unpickleSuccesses(infile = '/SuccessTable.pkl')
P10Successes = unpickleSuccesses(infile = '/SuccessTable.pkl')

nCond2LeftTrials = P1Successes.nTrials[0] + P2Successes.nTrials[0]+ P3Successes.nTrials[0]+ P4Successes.nTrials[0]+ P5Successes.nTrials[0]+ P6Successes.nTrials[0]+ P7Successes.nTrials[0]+ P8Successes.nTrials[0]+ P9Successes.nTrials[0]
nCond2RightTrials = P1Successes.nTrials[1] + P2Successes.nTrials[1]+ P3Successes.nTrials[1]+ P4Successes.nTrials[1]+ P5Successes.nTrials[1]+ P6Successes.nTrials[1]+ P7Successes.nTrials[1]+ P8Successes.nTrials[1]+ P9Successes.nTrials[1]

nCond3LeftTrials = P1Successes.nTrials[2] + P2Successes.nTrials[2]+ P3Successes.nTrials[2]+ P4Successes.nTrials[2]+ P5Successes.nTrials[2]+ P6Successes.nTrials[2]+ P7Successes.nTrials[2]+ P8Successes.nTrials[2]+ P9Successes.nTrials[2]
nCond3RightTrials = P1Successes.nTrials[3] + P2Successes.nTrials[3]+ P3Successes.nTrials[3]+ P4Successes.nTrials[3]+ P5Successes.nTrials[3]+ P6Successes.nTrials[3]+ P7Successes.nTrials[3]+ P8Successes.nTrials[3]+ P9Successes.nTrials[3]

nCond4LeftTrials = P1Successes.nTrials[4] + P2Successes.nTrials[4]+ P3Successes.nTrials[4]+ P4Successes.nTrials[4]+ P5Successes.nTrials[4]+ P6Successes.nTrials[4]+ P7Successes.nTrials[4]+ P8Successes.nTrials[4]+ P9Successes.nTrials[4]
nCond4RightTrials = P1Successes.nTrials[5] + P2Successes.nTrials[5]+ P3Successes.nTrials[5]+ P4Successes.nTrials[5]+ P5Successes.nTrials[5]+ P6Successes.nTrials[5]+ P7Successes.nTrials[5]+ P8Successes.nTrials[5]+ P9Successes.nTrials[5]

nCond5LeftTrials = P1Successes.nTrials[6] + P2Successes.nTrials[6]+ P3Successes.nTrials[6]+ P4Successes.nTrials[6]+ P5Successes.nTrials[6]+ P6Successes.nTrials[6]+ P7Successes.nTrials[6]+ P8Successes.nTrials[6]+ P9Successes.nTrials[6]
nCond5RightTrials = P1Successes.nTrials[7] + P2Successes.nTrials[7]+ P3Successes.nTrials[7]+ P4Successes.nTrials[7]+ P5Successes.nTrials[7]+ P6Successes.nTrials[7]+ P7Successes.nTrials[7]+ P8Successes.nTrials[7]+ P9Successes.nTrials[7]

nCond6LeftTrials = P1Successes.nTrials[8] + P2Successes.nTrials[8]+ P3Successes.nTrials[8]+ P4Successes.nTrials[8]+ P5Successes.nTrials[8]+ P6Successes.nTrials[8]+ P7Successes.nTrials[8]+ P8Successes.nTrials[8]+ P9Successes.nTrials[8]
nCond6RightTrials = P1Successes.nTrials[9] + P2Successes.nTrials[9]+ P3Successes.nTrials[9]+ P4Successes.nTrials[9]+ P5Successes.nTrials[9]+ P6Successes.nTrials[9]+ P7Successes.nTrials[9]+ P8Successes.nTrials[9]+ P9Successes.nTrials[9]

nCond7LeftTrials = P1Successes.nTrials[10] + P2Successes.nTrials[10]+ P3Successes.nTrials[10]+ P4Successes.nTrials[10]+ P5Successes.nTrials[10]+ P6Successes.nTrials[10]+ P7Successes.nTrials[10]+ P8Successes.nTrials[10]+ P9Successes.nTrials[10]
nCond7RightTrials = P1Successes.nTrials[11] + P2Successes.nTrials[11]+ P3Successes.nTrials[11]+ P4Successes.nTrials[11]+ P5Successes.nTrials[11]+ P6Successes.nTrials[11]+ P7Successes.nTrials[11]+ P8Successes.nTrials[11]+ P9Successes.nTrials[11]